In [1052]:
# _*_ coding: utf-8 _*_
# @Time: 2021/12/24 14:15 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 新华医院生脉注射液：1. 数据处理：201907041611133032包含在其他科原始数据中，它们和心内科用的受试者编号对应不同人
# @Description: 20220428, 有效性分析：住院时长。实验组：40，60；对照组。

# 数据处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()

## 导入预定义函数

In [3]:
# 字符串转换为时间格式
import datetime
def str_to_datetime(x):
    try:
        a = datetime.datetime.strptime(x, "%d/%m/%Y %H:%M:%S")
        return a
    except:
        return np.NaN

In [4]:
# 过滤特殊字符
def filter_spec_character(df,feature):
    df[feature]=df[feature].apply(lambda x: re.sub('<|>|\+|\/','',str(x)))
    return df

In [5]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [6]:
# 过滤异常大值
def filter_huge_value(df,feature):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    median_value=df[feature].median()
    if median_value == 0:
        pass
    else:
        df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else np.nan)
    return df

## 提取结局终点事件

In [7]:
'''
提取出院诊断-主要疾病转归：人数限定
'''

'\n提取出院诊断-主要疾病转归：人数限定\n'

In [8]:
# 其他科
df_oup_diagnosis_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='DS_出院情况',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

# 标记受试者编号
df_oup_diagnosis_1['受试者编号']=df_oup_diagnosis_1['受试者编号'].apply(lambda x: str(x)+'_a')

# 提取有效字段
df_oup_diagnosis_1=df_oup_diagnosis_1[['受试者编号','出院诊断','主要疾病转归_描述']]

In [9]:
df_oup_diagnosis_1.shape

(644, 3)

In [10]:
# 心内科
df_oup_diagnosis_2 = pd.read_excel(project_path+'/data/raw_data//导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='DS_出院情况',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

# 标记受试者编号
df_oup_diagnosis_2['受试者编号']=df_oup_diagnosis_2['受试者编号'].apply(lambda x: str(x)+'_b')
# 提取有效字段
df_oup_diagnosis_2=df_oup_diagnosis_2[['受试者编号','出院诊断','主要疾病转归_描述']]

In [11]:
df_oup_diagnosis_2.shape

(105, 3)

In [12]:
# 合并其他科和心内科联合用药
df_oup_diagnosis=pd.concat([df_oup_diagnosis_1,df_oup_diagnosis_2])
df_oup_diagnosis=df_oup_diagnosis[df_oup_diagnosis['主要疾病转归_描述'].notnull()]

In [13]:
df_oup_diagnosis.shape

(741, 3)

In [14]:
df_oup_diagnosis.head()

,受试者编号,出院诊断,主要疾病转归_描述
0,90001_a,大脑动脉狭窄脑梗死 NIHSS评分12分，高血压病（2级，很高危），2型糖尿病，脑梗死个人史...,好转
2,90003_a,I型呼吸衰竭 重症肺炎 肺栓塞(中低危)心功能不全 高血压病I级(高危)肝功能不全 下肢静脉...,致死
3,90004_a,"尿毒症,慢性肾小球肾炎,腹膜透析;胸腔积液;心房颤动伴II度房室传导阻滞,心功能III-IV...",好转
4,90005_a,"尿毒症,慢性肾小球肾炎, 腹膜透析,高血压病(3级 极高危);肺部感染,肾性贫血,甲状腺结节",好转
5,90006_a,"急性脑梗死(NIHSS评分2分);动脉斑块形成;高血压(3级,很高危);冠状动脉粥样硬化性心...",好转


In [15]:
df_oup_diagnosis['主要疾病转归_描述'].value_counts()

好转    671
致死     39
未愈     30
治愈      1
Name: 主要疾病转归_描述, dtype: int64

In [16]:
# 剔除致死患者
df_oup_diagnosis=df_oup_diagnosis[df_oup_diagnosis['主要疾病转归_描述']!='致死']

In [17]:
df_oup_diagnosis.shape

(702, 3)

In [18]:
df_oup_diagnosis['主要疾病转归_描述']=df_oup_diagnosis['主要疾病转归_描述'].apply(lambda x: 1 if re.match('致死|未愈',x) else 0)

In [19]:
df_oup_diagnosis['主要疾病转归_描述'].value_counts()

0    672
1     30
Name: 主要疾病转归_描述, dtype: int64

In [20]:
df_oup_diagnosis.head()

,受试者编号,出院诊断,主要疾病转归_描述
0,90001_a,大脑动脉狭窄脑梗死 NIHSS评分12分，高血压病（2级，很高危），2型糖尿病，脑梗死个人史...,0
3,90004_a,"尿毒症,慢性肾小球肾炎,腹膜透析;胸腔积液;心房颤动伴II度房室传导阻滞,心功能III-IV...",0
4,90005_a,"尿毒症,慢性肾小球肾炎, 腹膜透析,高血压病(3级 极高危);肺部感染,肾性贫血,甲状腺结节",0
5,90006_a,"急性脑梗死(NIHSS评分2分);动脉斑块形成;高血压(3级,很高危);冠状动脉粥样硬化性心...",0
6,90007_a,"急性脑梗死可能(NIHSS评分0分);高血压(3级,很高危);泌尿道感染;脂肪肝;甲状腺结节...",0


In [21]:
df_oup_diagnosis.to_excel(project_path+'/data/processed_data/df_1.3_提取出院诊断转归数据.xlsx')

## 提取用药信息

### 其他科用药数据

In [22]:
drug_data_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='EX_生脉注射液用药情况',
#                            header=1,  # 指定第2行作为标题行
                           skiprows=[1,1,1])  # 跳过第1行

In [23]:
print(drug_data_1.shape)
print(drug_data_1['SUBJECTID'].nunique())

(1107, 30)
645


In [24]:
drug_data_1.head()

,STUDYID,SITEID,SITE,SUBJECTID,VISIT,EXYN,EXYN_CHAR,EXSTDAT,EXENDAT,EXDLDAYS,...,EXDSTXTUT,EXDSTXTSD,EXDSTXTSU,EXMENS,EXMENS_CHAR,EXMENOTH,EXCOMDYN,EXCOMDYN_CHAR,ENTSTDTC,ENTENDTC
0,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90001,生脉注射液用药情况,1.0,是,2018-04-07,2018-04-15,9.0,...,ml,NaN,NaN,99.0,其它,0.9% NS 100ml,2.0,无,2018/4/20 15:59:03,NaN
1,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90001,生脉注射液用药情况,2.0,否,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/8/1 23:14:03,NaN
2,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90002,生脉注射液用药情况,2.0,否,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/10/1 19:20:27,NaN
3,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003,生脉注射液用药情况,1.0,是,2018-03-12,2018-03-17,6.0,...,ml,NaN,NaN,1.0,5% GS 250ml,NaN,2.0,无,2018/9/16 16:37:59,NaN
4,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003,生脉注射液用药情况,2.0,否,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/8/8 16:31:10,NaN


In [25]:
# 提取使用生脉注射液的数据
drug_data_1=drug_data_1[drug_data_1['EXYN_CHAR']=='是']
drug_data_1=drug_data_1.reset_index(drop=True)

In [26]:
print(drug_data_1.shape)
print(drug_data_1['SUBJECTID'].nunique())

(635, 30)
635


In [27]:
# 删除用药剂量为空的数据，频次：EXDOSFRQ；单次剂量：EXDSTXT
drug_data_1=drug_data_1[(drug_data_1['EXDOSFRQ'].notnull()) &  
                       (drug_data_1['EXDSTXT'].notnull())]
drug_data_1=drug_data_1.reset_index(drop=True)

In [28]:
print(drug_data_1.shape)
print(drug_data_1['SUBJECTID'].nunique())

(635, 30)
635


In [29]:
# 提取有效字段
drug_data_1=drug_data_1[['SUBJECTID','EXSTDAT','EXENDAT','EXDLDAYS','EXDOSFRQ','EXDSTXT']]

In [30]:
# 标记受试者编号
drug_data_1['SUBJECTID']=drug_data_1['SUBJECTID'].apply(lambda x: str(x)+'_a')

In [31]:
drug_data_1.head()

,SUBJECTID,EXSTDAT,EXENDAT,EXDLDAYS,EXDOSFRQ,EXDSTXT
0,90001_a,2018-04-07,2018-04-15,9.0,1.0,30.0
1,90003_a,2018-03-12,2018-03-17,6.0,1.0,40.0
2,90004_a,2018-02-13,2018-02-13,1.0,1.0,40.0
3,90005_a,2018-02-09,2018-02-09,1.0,1.0,40.0
4,90006_a,2018-02-23,2018-02-23,1.0,1.0,30.0


In [32]:
drug_data_1.to_excel(project_path+'/data/processed_data/用药数据/其他科_drug_1.xlsx')

### 心内科用药数据

In [33]:
drug_data_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='EX_生脉注射液用药情况',
                            #                            header=1,  # 指定第2行作为标题行
                           skiprows=[1,1,1])  # 跳过第几行)

In [34]:
print(drug_data_2.shape)
print(drug_data_2['SUBJECTID'].nunique())

(192, 30)
105


In [35]:
# 提取使用生脉注射液的数据
drug_data_2=drug_data_2[drug_data_2['EXYN_CHAR']=='是']
drug_data_2=drug_data_2.reset_index(drop=True)

In [36]:
print(drug_data_2.shape)
print(drug_data_2['SUBJECTID'].nunique())

(105, 30)
105


In [37]:
# 删除用药剂量为空的数据
drug_data_2=drug_data_2[(drug_data_2['EXDOSFRQ'].notnull()) &
                       (drug_data_2['EXDSTXT'].notnull())]
drug_data_2=drug_data_2.reset_index(drop=True)

In [38]:
print(drug_data_2.shape)
print(drug_data_2['SUBJECTID'].nunique())

(105, 30)
105


In [39]:
# 提取有效字段
drug_data_2=drug_data_2[['SUBJECTID','EXSTDAT','EXENDAT','EXDLDAYS','EXDOSFRQ','EXDSTXT']]

In [1091]:
# 标记受试者编号
drug_data_2['SUBJECTID']=drug_data_2['SUBJECTID'].apply(lambda x: str(x)+'_b')

In [1092]:
drug_data_2.head()

,SUBJECTID,EXSTDAT,EXENDAT,EXDLDAYS,EXDOSFRQ,EXDSTXT
0,90001_b,2018-02-13,2018-02-13,1,1.0,40.0
1,90002_b,2019-04-24,2019-05-05,12,1.0,60.0
2,90003_b,2019-05-24,NaN,1,1.0,40.0
3,90004_b,2019-04-01,2019-04-03,3,1.0,60.0
4,90005_b,2019-05-20,2019-05-27,8,1.0,30.0


In [1093]:
drug_data_2.to_excel(project_path+'/data/processed_data/用药数据/心内科_drug_2.xlsx')

### 计算日剂量

In [1094]:
# 合并其他科和心内科用药数据
df_SUP_drug=pd.concat([drug_data_1,drug_data_2],axis=0)

In [1095]:
#计算日剂量
df_SUP_drug['日剂量']=df_SUP_drug['EXDOSFRQ'].astype('float') * df_SUP_drug['EXDSTXT'].astype('float')

In [1096]:
print(df_SUP_drug.shape)
print(df_SUP_drug['SUBJECTID'].nunique())

(740, 7)
740


In [1097]:
df_SUP_drug.columns

Index(['SUBJECTID', 'EXSTDAT', 'EXENDAT', 'EXDLDAYS', 'EXDOSFRQ', 'EXDSTXT',
       '日剂量'],
      dtype='object')

In [1098]:
df_SUP_drug.rename(columns={'SUBJECTID':'受试者编号',
                            'EXSTDAT':'用药开始时间',
                            'EXENDAT':'用药结束时间',
                            'EXDLDAYS':'用药天数'},inplace=True)
df_SUP_drug=df_SUP_drug.reset_index(drop=True)

In [1099]:
# 提取含有结局终点事件的生脉饮用药
df_SUP_drug=pd.merge(df_oup_diagnosis,df_SUP_drug,on='受试者编号',how='inner')

In [1100]:
df_SUP_drug.shape

(695, 9)

In [1101]:
df_SUP_drug.EXDOSFRQ.unique()

array([1., 2.])

In [1102]:
df_SUP_drug.head()

,受试者编号,出院诊断,主要疾病转归_描述,用药开始时间,用药结束时间,用药天数,EXDOSFRQ,EXDSTXT,日剂量
0,90001_a,大脑动脉狭窄脑梗死 NIHSS评分12分，高血压病（2级，很高危），2型糖尿病，脑梗死个人史...,0,2018-04-07,2018-04-15,9.0,1.0,30.0,30.0
1,90004_a,"尿毒症,慢性肾小球肾炎,腹膜透析;胸腔积液;心房颤动伴II度房室传导阻滞,心功能III-IV...",0,2018-02-13,2018-02-13,1.0,1.0,40.0,40.0
2,90005_a,"尿毒症,慢性肾小球肾炎, 腹膜透析,高血压病(3级 极高危);肺部感染,肾性贫血,甲状腺结节",0,2018-02-09,2018-02-09,1.0,1.0,40.0,40.0
3,90006_a,"急性脑梗死(NIHSS评分2分);动脉斑块形成;高血压(3级,很高危);冠状动脉粥样硬化性心...",0,2018-02-23,2018-02-23,1.0,1.0,30.0,30.0
4,90007_a,"急性脑梗死可能(NIHSS评分0分);高血压(3级,很高危);泌尿道感染;脂肪肝;甲状腺结节...",0,2018-02-23,2018-02-23,1.0,1.0,30.0,30.0


In [1103]:
df_SUP_drug.to_excel(project_path+'/data/processed_data/df_1.4.3_计算生脉用药_日剂量.xlsx')

In [1104]:
df_SUP_drug.日剂量.value_counts()

40.0     388
60.0     173
30.0     101
50.0      20
20.0      10
10.0       2
120.0      1
Name: 日剂量, dtype: int64

### 筛选高低剂量组用药

In [1105]:
'''
只保留40和60mg，用于高低剂量组分析
'''

'\n只保留40和60mg，用于高低剂量组分析\n'

In [1106]:
df_SUP_drug.日剂量.value_counts()

40.0     388
60.0     173
30.0     101
50.0      20
20.0      10
10.0       2
120.0      1
Name: 日剂量, dtype: int64

In [1107]:
# 保留高低剂量组
df_SUP_drug = df_SUP_drug[(df_SUP_drug.日剂量==40) | (df_SUP_drug.日剂量==60)]

In [1108]:
df_SUP_drug.日剂量.value_counts()

40.0    388
60.0    173
Name: 日剂量, dtype: int64

In [1109]:
df_SUP_drug.to_csv(project_path+'/data/processed_data/df_1.4.4_筛选高低剂量组用药.csv')

### 计算用药时长

In [1110]:
aaa=df_SUP_drug[df_SUP_drug['用药结束时间'].isnull()]
bbb=df_SUP_drug[df_SUP_drug['用药结束时间'].notnull()]
aaa['用药结束时间']=aaa['用药开始时间']
df_SUP_drug=pd.concat([aaa,bbb],axis=0)
df_SUP_drug=df_SUP_drug.sort_values(by=['受试者编号','用药开始时间'])
df_SUP_drug=df_SUP_drug.reset_index(drop=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1111]:
df_SUP_drug['用药天数']=(pd.to_datetime(df_SUP_drug['用药结束时间']) - pd.to_datetime(df_SUP_drug['用药开始时间']))
df_SUP_drug['用药天数']=df_SUP_drug['用药天数'].apply(lambda x:x.days+1)

In [1112]:
df_SUP_drug.shape

(561, 9)

In [1113]:
df_SUP_drug.to_excel(project_path+'/data/processed_data/df_1.4.5_用药时长.xlsx')

In [1114]:
df_SUP_drug[df_SUP_drug['用药天数']>3].shape

(444, 9)

## 纳入冠心病和心梗

### 提取疾病

In [1115]:
df_diagnose_1=pd.read_excel(project_path + '/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',sheet_name='RC_西医诊断',header=1)

df_diagnose_2=pd.read_excel(project_path + '/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',sheet_name='RC_西医诊断',header=1)

df_diagnose_1['受试者编号']=df_diagnose_1['受试者编号'].astype('str')+'_'+'a'
df_diagnose_2['受试者编号']=df_diagnose_2['受试者编号'].astype('str')+'_'+'b'
df_diagnose=pd.concat([df_diagnose_1,df_diagnose_2],axis=0)
df_diagnose=df_diagnose.reset_index(drop=True)

In [1116]:
print(df_diagnose_1.shape)
print(df_diagnose_2.shape)

(699, 10)
(104, 10)


In [1117]:
df_diagnose=df_diagnose[df_diagnose['西医诊断'].notnull()]
df_diagnose=df_diagnose.reset_index(drop=True)

In [1118]:
df_diagnose.shape

(799, 10)

In [1119]:
df_diagnose.head()

,方案号,中心标识,中心名称,受试者编号,访视,编号,西医诊断,开始日期,原始录入日期,最后修改日期
0,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90001_a,入组访视,1,大脑动脉狭窄脑梗死，高血压病，2DM，脑梗个人史，动脉斑块形成，双侧甲状腺结节,2018-04-03,2018/4/20 15:04:29,NaN
1,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003_a,入组访视,1,肺栓塞(中低危),2018-01-31,2018/9/16 15:44:19,NaN
2,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003_a,入组访视,2,两肺阴影待查,2018-01-31,2018/9/16 15:44:45,NaN
3,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003_a,入组访视,3,高血压病1级(高危),2018-01-31,2018/9/16 15:45:10,NaN
4,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003_a,入组访视,4,肝功能不全,2018-01-31,2018/9/16 15:45:39,NaN


In [1120]:
df_diagnose['西医诊断']=df_diagnose['西医诊断'].astype('str').apply(lambda x: re.split(r'[,，：: ?？;；/(/)（）]',x))

In [1121]:
df_diagnose=df_diagnose[['受试者编号','西医诊断']]

In [1122]:
temp_list=[]
for i in df_diagnose['受试者编号'].unique():
#     print(i)
    temp=df_diagnose[df_diagnose['受试者编号']==i]
    temp=temp.reset_index(drop=True)
    if temp.shape[0]>1:
        for j in range(1,temp.shape[0]):
            temp.loc[0,'西医诊断']=temp.loc[0,'西医诊断'] + temp.loc[j,'西医诊断']
        temp=temp.loc[:0,]
    temp_list.append(temp)

In [1123]:
df_diagnose=pd.concat(temp_list,axis=0).sort_values(['受试者编号']).reset_index(drop=True)
del temp_list

In [1124]:
df_diagnose.head()

,受试者编号,西医诊断
0,90001_a,"[大脑动脉狭窄脑梗死, 高血压病, 2DM, 脑梗个人史, 动脉斑块形成, 双侧甲状腺结节]"
1,90002_b,"[急性前壁心肌梗塞, 冠心病killip, Ⅰ级, 急诊LAD-PCI术, , 2型糖尿病]"
2,90003_a,"[肺栓塞, 中低危, , 两肺阴影待查, 高血压病1级, 高危, , 肝功能不全, 腔隙性脑梗塞]"
3,90003_b,"[, 冠状动脉造影硬化性心脏病, 急性冠脉综合征, 高血压病]"
4,90004_a,"[尿毒症, 慢性肾小球肾炎, 腹膜透析, 胸腔积液, 心房颤动伴II度房室传导阻滞, 心功能..."


In [1125]:
# 2022.1.12，重新梳理合并疾病
'冠心病','冠状动脉粥样硬化性心脏病','冠状动脉粥样硬化','冠心病kp'
'肺部感染'
'糖尿病','2型糖尿'
'高血压'
'心功能不全','心功能'
'脑梗死','脑梗'
'头晕'
'急性冠脉综合症','急性冠脉综合征'
'房颤'
'肾功能不全','肾功能'
'尿毒症'
'脓毒血症'
'肺水肿'
'肺气肿'
'呼吸衰竭','呼衰'
'胸腔积液'
'尿路感染'
'腹膜透析'
'肝功能不全','肝功'
'高脂血症'
'慢性支气管炎'
'胆囊结石','胆结石'
'早搏'
'急性肾损伤'
'心肌梗塞','心肌梗死','心梗'

('心肌梗塞', '心肌梗死', '心梗')

In [1126]:
df_diagnose['西医诊断_raw']=df_diagnose['西医诊断']

In [1127]:
df_diagnose['西医诊断']

0         [大脑动脉狭窄脑梗死, 高血压病, 2DM, 脑梗个人史, 动脉斑块形成, 双侧甲状腺结节]
1         [急性前壁心肌梗塞, 冠心病killip, Ⅰ级, 急诊LAD-PCI术, , 2型糖尿病]
2      [肺栓塞, 中低危, , 两肺阴影待查, 高血压病1级, 高危, , 肝功能不全, 腔隙性脑梗塞]
3                        [, 冠状动脉造影硬化性心脏病, 急性冠脉综合征, 高血压病]
4      [尿毒症, 慢性肾小球肾炎, 腹膜透析, 胸腔积液, 心房颤动伴II度房室传导阻滞, 心功能...
                             ...                        
734    [帕金森综合征, , , , , , , , , , , , 高血压2级, 高危, , , ...
735                    [头痛背痛待查, 急性冠脉综合征, , , 高血压3级, 很高危]
736         [横纹肌溶解, 2型糖尿病, 肝功能受损, 反流性食管炎, 糜烂性全胃炎, 血管性痴呆]
737                                      [心肌损伤, 化脓性扁桃体炎]
738        [初步诊断, , , 帕金森病, 高血压病, 3级, 很高危, , 白内障, 脑梗个人史]
Name: 西医诊断, Length: 739, dtype: object

In [1128]:
for i in range(df_diagnose.shape[0]):
    df_diagnose.loc[i,'西医诊断']=[x.replace('冠状动脉粥样硬化','冠心病') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('糖尿','糖尿病') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('心功能','心功能不全') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('心律','心功能不全') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('心力衰竭','心功能不全') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('脑梗','脑梗死') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('急性冠脉综合症','急性冠脉综合征') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('肾功能','肾功能不全') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('呼衰','呼吸衰竭') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('肝功能','肝功能不全') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('胆结石','胆囊结石') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('心肌梗死','心肌梗塞') for x in df_diagnose.loc[i,'西医诊断']]
    df_diagnose.loc[i,'西医诊断']=[x.replace('心梗','心肌梗塞') for x in df_diagnose.loc[i,'西医诊断']]

In [1129]:
df_diagnose.to_excel(project_path+'/data/processed_data/df_1.4.1_西医诊断疾病.xlsx')

### 疾病转置

In [1130]:
disease_name=['冠心病','肺部感染','糖尿病','高血压','心功能不全','脑梗死','头晕','急性冠脉综合征','房颤',\
               '肾功能不全','尿毒症','脓毒血症','肺水肿','肺气肿','呼吸衰竭','胸腔积液','尿路感染','腹膜透析',\
              '肝功能不全','高脂血症','慢性支气管炎','胆囊结石','早搏','急性肾损伤','心肌梗塞']

In [1131]:
len(disease_name)

25

In [1132]:
df_disease=df_diagnose.copy()
for i in range(df_disease.shape[0]):
    diag_str=';'.join(df_disease.loc[i,'西医诊断'])
    for j in disease_name:
        if j in diag_str:
            df_disease.loc[i,j]=1
        else:
            df_disease.loc[i,j]=0

In [1133]:
df_disease.shape

(739, 28)

In [1134]:
df_disease.head()

,受试者编号,西医诊断,西医诊断_raw,冠心病,肺部感染,糖尿病,高血压,心功能不全,脑梗死,头晕,...,胸腔积液,尿路感染,腹膜透析,肝功能不全,高脂血症,慢性支气管炎,胆囊结石,早搏,急性肾损伤,心肌梗塞
0,90001_a,"[大脑动脉狭窄脑梗死死, 高血压病, 2DM, 脑梗死个人史, 动脉斑块形成, 双侧甲状腺结节]","[大脑动脉狭窄脑梗死, 高血压病, 2DM, 脑梗个人史, 动脉斑块形成, 双侧甲状腺结节]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90002_b,"[急性前壁心肌梗塞, 冠心病killip, Ⅰ级, 急诊LAD-PCI术, , 2型糖尿病病]","[急性前壁心肌梗塞, 冠心病killip, Ⅰ级, 急诊LAD-PCI术, , 2型糖尿病]",1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,90003_a,"[肺栓塞, 中低危, , 两肺阴影待查, 高血压病1级, 高危, , 肝功能不全不全, 腔隙...","[肺栓塞, 中低危, , 两肺阴影待查, 高血压病1级, 高危, , 肝功能不全, 腔隙性脑梗塞]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90003_b,"[, 冠状动脉造影硬化性心脏病, 急性冠脉综合征, 高血压病]","[, 冠状动脉造影硬化性心脏病, 急性冠脉综合征, 高血压病]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,90004_a,"[尿毒症, 慢性肾小球肾炎, 腹膜透析, 胸腔积液, 心房颤动伴II度房室传导阻滞, 心功能...","[尿毒症, 慢性肾小球肾炎, 腹膜透析, 胸腔积液, 心房颤动伴II度房室传导阻滞, 心功能...",0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1135]:
df_disease.to_excel(project_path+'/data/processed_data/df_1.5.2_疾病转置.xlsx')

In [1136]:
df_disease.columns

Index(['受试者编号', '西医诊断', '西医诊断_raw', '冠心病', '肺部感染', '糖尿病', '高血压', '心功能不全',
       '脑梗死', '头晕', '急性冠脉综合征', '房颤', '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿',
       '呼吸衰竭', '胸腔积液', '尿路感染', '腹膜透析', '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏',
       '急性肾损伤', '心肌梗塞'],
      dtype='object')

### 疾病归类

In [1137]:
df_disease_class=df_disease[['受试者编号','冠心病', '肺部感染', '糖尿病', '高血压', '心功能不全', '脑梗死', '头晕', '急性冠脉综合征',
                         '房颤', '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿', '呼吸衰竭', '胸腔积液', '尿路感染',
                         '腹膜透析', '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏',
                         '急性肾损伤', '心肌梗塞']]

In [1138]:
'''
心血管疾病：高血压、冠心病、心肌梗塞、急性冠脉综合征、心功能不全、房颤、早搏
呼吸系统疾病：肺部感染、呼吸衰竭、胸腔积液、肺水肿、慢性支气管炎、肺气肿
神经系统疾病：脑梗死、头晕
泌尿系统疾病：肾功能不全、急性肾损伤、尿毒症、腹膜透析、尿路感染
内分泌和代谢疾病：糖尿病、高脂血症
消化系统疾病：肝功能不全、胆囊结石
危重症疾病：脓毒血症
'''

'\n心血管疾病：高血压、冠心病、心肌梗塞、急性冠脉综合征、心功能不全、房颤、早搏\n呼吸系统疾病：肺部感染、呼吸衰竭、胸腔积液、肺水肿、慢性支气管炎、肺气肿\n神经系统疾病：脑梗死、头晕\n泌尿系统疾病：肾功能不全、急性肾损伤、尿毒症、腹膜透析、尿路感染\n内分泌和代谢疾病：糖尿病、高脂血症\n消化系统疾病：肝功能不全、胆囊结石\n危重症疾病：脓毒血症\n'

In [1139]:
for i in range(df_disease_class.shape[0]):
    # 心血管疾病
    if df_disease_class.loc[i,['高血压','冠心病','心肌梗塞','急性冠脉综合征','心功能不全','房颤','早搏']].sum() >=1:
        df_disease_class.loc[i,'心血管疾病_label']=1
    else:
        df_disease_class.loc[i,'心血管疾病_label']=0
     # 呼吸系统疾病
    if df_disease_class.loc[i,['肺部感染','呼吸衰竭','胸腔积液','肺水肿','慢性支气管炎','肺气肿']].sum() >=1:
        df_disease_class.loc[i,'呼吸系统疾病_label']=1
    else:
        df_disease_class.loc[i,'呼吸系统疾病_label']=0
     # 神经系统疾病
    if df_disease_class.loc[i,['脑梗死','头晕']].sum() >=1:
        df_disease_class.loc[i,'神经系统疾病_label']=1
    else:
        df_disease_class.loc[i,'神经系统疾病_label']=0
     # 泌尿系统疾病
    if df_disease_class.loc[i,['肾功能不全','急性肾损伤','尿毒症','腹膜透析','尿路感染']].sum() >=1:
        df_disease_class.loc[i,'泌尿系统疾病_label']=1
    else:
        df_disease_class.loc[i,'泌尿系统疾病_label']=0
     # 内分泌和代谢疾病
    if df_disease_class.loc[i,['糖尿病','高脂血症']].sum() >=1:
        df_disease_class.loc[i,'内分泌和代谢疾病_label']=1
    else:
        df_disease_class.loc[i,'内分泌和代谢疾病_label']=0
     # 消化系统疾病
    if df_disease_class.loc[i,['肝功能不全','胆囊结石']].sum() >=1:
        df_disease_class.loc[i,'消化系统疾病_label']=1
    else:
        df_disease_class.loc[i,'消化系统疾病_label']=0
     # 危重症疾病
    if df_disease_class.loc[i,['脓毒血症']].sum() >=1:
        df_disease_class.loc[i,'危重症疾病_label']=1
    else:
        df_disease_class.loc[i,'危重症疾病_label']=0

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [1140]:
df_disease_class.shape

(739, 33)

In [1141]:
# 删除细分疾病变量
# df_disease_class=df_disease_class.drop(disease_name,axis=1)

In [1142]:
df_disease_class.shape

(739, 33)

In [1143]:
df_disease_class.head()

,受试者编号,冠心病,肺部感染,糖尿病,高血压,心功能不全,脑梗死,头晕,急性冠脉综合征,房颤,...,早搏,急性肾损伤,心肌梗塞,心血管疾病_label,呼吸系统疾病_label,神经系统疾病_label,泌尿系统疾病_label,内分泌和代谢疾病_label,消化系统疾病_label,危重症疾病_label
0,90001_a,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,90002_b,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,90003_a,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,90003_b,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,90004_a,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [1144]:
df_disease_class.to_excel(project_path+'/data/processed_data/df_1.5.3_疾病归类.xlsx')

### 并入疾病

In [1145]:
df_SUP_disease=pd.merge(df_SUP_drug,df_disease_class,on='受试者编号',how='left')

In [1146]:
df_SUP_disease.shape

(561, 41)

In [1147]:
df_SUP_disease.isnull().any()

受试者编号             False
出院诊断               True
主要疾病转归_描述         False
用药开始时间            False
用药结束时间            False
用药天数              False
EXDOSFRQ          False
EXDSTXT           False
日剂量               False
冠心病                True
肺部感染               True
糖尿病                True
高血压                True
心功能不全              True
脑梗死                True
头晕                 True
急性冠脉综合征            True
房颤                 True
肾功能不全              True
尿毒症                True
脓毒血症               True
肺水肿                True
肺气肿                True
呼吸衰竭               True
胸腔积液               True
尿路感染               True
腹膜透析               True
肝功能不全              True
高脂血症               True
慢性支气管炎             True
胆囊结石               True
早搏                 True
急性肾损伤              True
心肌梗塞               True
心血管疾病_label        True
呼吸系统疾病_label       True
神经系统疾病_label       True
泌尿系统疾病_label       True
内分泌和代谢疾病_label     True
消化系统疾病_label       True
危重症疾病_label        True
dtype: bool

In [1148]:
df_SUP_disease.to_excel(project_path+'/data/processed_data/df_1.5.4_并入疾病.xlsx')

In [1149]:
df_SUP_disease['日剂量'].value_counts()

40.0    388
60.0    173
Name: 日剂量, dtype: int64

### 统计疾病

In [1150]:
'''
心血管疾病：高血压、冠心病、心肌梗塞、急性冠脉综合征、心功能不全、房颤、早搏
呼吸系统疾病：肺部感染、呼吸衰竭、胸腔积液、肺水肿、慢性支气管炎、肺气肿
神经系统疾病：脑梗死、头晕
泌尿系统疾病：肾功能不全、急性肾损伤、尿毒症、腹膜透析、尿路感染
内分泌和代谢疾病：糖尿病、高脂血症
消化系统疾病：肝功能不全、胆囊结石
危重症疾病：脓毒血症
'''

'\n心血管疾病：高血压、冠心病、心肌梗塞、急性冠脉综合征、心功能不全、房颤、早搏\n呼吸系统疾病：肺部感染、呼吸衰竭、胸腔积液、肺水肿、慢性支气管炎、肺气肿\n神经系统疾病：脑梗死、头晕\n泌尿系统疾病：肾功能不全、急性肾损伤、尿毒症、腹膜透析、尿路感染\n内分泌和代谢疾病：糖尿病、高脂血症\n消化系统疾病：肝功能不全、胆囊结石\n危重症疾病：脓毒血症\n'

In [1151]:
disease_class_list=['心血管疾病','呼吸系统疾病','神经系统疾病','泌尿系统疾病','内分泌和代谢疾病','消化系统疾病','危重症疾病']

In [1152]:
for i in disease_class_list:
    print(df_SUP_disease[i+'_label'].value_counts())

1.0    433
0.0    122
Name: 心血管疾病_label, dtype: int64
0.0    381
1.0    174
Name: 呼吸系统疾病_label, dtype: int64
0.0    395
1.0    160
Name: 神经系统疾病_label, dtype: int64
0.0    479
1.0     76
Name: 泌尿系统疾病_label, dtype: int64
0.0    397
1.0    158
Name: 内分泌和代谢疾病_label, dtype: int64
0.0    527
1.0     28
Name: 消化系统疾病_label, dtype: int64
0.0    534
1.0     21
Name: 危重症疾病_label, dtype: int64


### 提取冠心病和心梗患者

In [1153]:
# 删除空值
df_SUP_disease=df_SUP_disease[df_SUP_disease['冠心病'].notnull()]
df_SUP_disease=df_SUP_disease[df_SUP_disease['心肌梗塞'].notnull()]
# 提取冠心病和心梗患者
df_SUP_disease_select=df_SUP_disease[(df_SUP_disease['冠心病']==1) | (df_SUP_disease['心肌梗塞']==1)]

In [1154]:
df_SUP_disease_select.shape

(222, 41)

In [1155]:
df_SUP_disease_select.to_excel(project_path+'/data/processed_data/df_1.5.6_提取冠心病和心梗患者.xlsx')

In [1156]:
# ### 提取心血管疾病患者

# df_SUP_disease_select=df_SUP_disease[df_SUP_disease['心血管疾病_label']==1]

# df_SUP_disease_select.shape

# df_SUP_disease_select.to_excel(project_path+'/data/processed_data/df_1.5.6_提取心血管疾病患者.xlsx')

## 统计用药时长和出入院时长

### 提取入院时间

In [1157]:
# 其他科 VD_住院日期
df_入院时间_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='VD_住院日期',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

df_入院时间_1['受试者编号']=df_入院时间_1['受试者编号'].apply(lambda x: str(x)+'_a')
df_入院时间_1=df_入院时间_1[['受试者编号','住院日期']]

In [1158]:
df_入院时间_1.shape

(645, 2)

In [1159]:
# 心内科入院时间
df_入院时间_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='VD_住院日期',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)

df_入院时间_2['受试者编号']=df_入院时间_2['受试者编号'].apply(lambda x: str(x)+'_b')
df_入院时间_2=df_入院时间_2[['受试者编号','住院日期']]

In [1160]:
df_入院时间_2.shape

(105, 2)

In [1161]:
df_入院时间=pd.concat([df_入院时间_1,df_入院时间_2],axis=0)

In [1162]:
df_入院时间.head()

,受试者编号,住院日期
0,90001_a,2018-04-03
1,90002_a,2018-07-17
2,90003_a,2018-01-31
3,90004_a,2018-01-26
4,90005_a,2018-02-07


In [1163]:
df_入院时间.shape

(750, 2)

In [1164]:
df_入院时间.to_excel(project_path+'/data/processed_data/df_1.6.1_提取入院时间.xlsx')

### 提取出院时间

In [1165]:
# 其他科 VD_出院日期
df_出院时间_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='DS_出院情况',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

df_出院时间_1['受试者编号']=df_出院时间_1['受试者编号'].apply(lambda x: str(x)+'_a')
df_出院时间_1=df_出院时间_1[['受试者编号','完成日期']]

In [1166]:
df_出院时间_1.shape

(644, 2)

In [1167]:
# 心内科出院时间
df_出院时间_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='DS_出院情况',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)

df_出院时间_2['受试者编号']=df_出院时间_2['受试者编号'].apply(lambda x: str(x)+'_b')
df_出院时间_2=df_出院时间_2[['受试者编号','完成日期']]

In [1168]:
df_出院时间_2.shape

(105, 2)

In [1169]:
df_出院时间 = pd.concat([df_出院时间_1,df_出院时间_2],axis=0)

In [1170]:
df_出院时间.shape

(749, 2)

In [1171]:
df_出院时间.to_excel(project_path+'/data/processed_data/df_1.6.2_提取出院时间.xlsx')

### 合并住院时间

In [1172]:
df_住院时间 = pd.merge(df_入院时间,df_出院时间,on='受试者编号',how='left')

In [1173]:
df_住院时间.shape

(750, 3)

In [1174]:
df_住院时间.columns

Index(['受试者编号', '住院日期', '完成日期'], dtype='object')

In [1175]:
df_住院时间.to_excel(project_path+'/data/processed_data/df_1.6.3_合并住院时间.xlsx')

### 并入用药-计算住院时长

In [1176]:
# 并入住院时间
df_SUP_inp=pd.merge(df_SUP_disease_select,df_住院时间,on='受试者编号',how='left')

In [1177]:
df_SUP_inp.shape

(222, 43)

In [1178]:
df_SUP_inp.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病', '肺部感染', '糖尿病', '高血压', '心功能不全', '脑梗死', '头晕',
       '急性冠脉综合征', '房颤', '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿', '呼吸衰竭', '胸腔积液',
       '尿路感染', '腹膜透析', '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏', '急性肾损伤',
       '心肌梗塞', '心血管疾病_label', '呼吸系统疾病_label', '神经系统疾病_label', '泌尿系统疾病_label',
       '内分泌和代谢疾病_label', '消化系统疾病_label', '危重症疾病_label', '住院日期', '完成日期'],
      dtype='object')

In [1179]:
# 补充住院时间
aaa=df_SUP_inp[df_SUP_inp['住院日期'].isnull()]
bbb=df_SUP_inp[df_SUP_inp['住院日期'].notnull()]
aaa['住院日期']=aaa['用药开始时间']
df_SUP_inp=pd.concat([aaa,bbb],axis=0)
df_SUP_inp=df_SUP_inp.sort_values(by=['受试者编号','用药开始时间'])
df_SUP_inp=df_SUP_inp.reset_index(drop=True)

In [1180]:
df_SUP_inp.to_excel(project_path+'/data/processed_data/df_1.6.4_temp_补充入院时间.xlsx')

In [1181]:
df_SUP_inp.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病', '肺部感染', '糖尿病', '高血压', '心功能不全', '脑梗死', '头晕',
       '急性冠脉综合征', '房颤', '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿', '呼吸衰竭', '胸腔积液',
       '尿路感染', '腹膜透析', '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏', '急性肾损伤',
       '心肌梗塞', '心血管疾病_label', '呼吸系统疾病_label', '神经系统疾病_label', '泌尿系统疾病_label',
       '内分泌和代谢疾病_label', '消化系统疾病_label', '危重症疾病_label', '住院日期', '完成日期'],
      dtype='object')

In [1182]:
# 补充出院时间
aaa=df_SUP_inp[df_SUP_inp['完成日期'].isnull()]
bbb=df_SUP_inp[df_SUP_inp['完成日期'].notnull()]
aaa['完成日期']=aaa['用药结束时间']
df_SUP_inp=pd.concat([aaa,bbb],axis=0)
df_SUP_inp=df_SUP_inp.sort_values(by=['受试者编号','用药开始时间'])
df_SUP_inp=df_SUP_inp.reset_index(drop=True)

In [1183]:
df_SUP_inp.to_excel(project_path+'/data/processed_data/df_1.6.4_temp_补充出院时间.xlsx')

In [1184]:
df_SUP_inp['住院时长']=(pd.to_datetime(df_SUP_inp['完成日期']) - pd.to_datetime(df_SUP_inp['住院日期']))
df_SUP_inp['住院时长']=df_SUP_inp['住院时长'].apply(lambda x:x.days+1)

In [1185]:
df_SUP_inp.shape

(222, 44)

In [1186]:
df_SUP_inp.to_excel(project_path+'/data/processed_data/df_1.6.4_计算住院时长.xlsx')

In [505]:
df_SUP_inp=pd.read_excel(project_path+'/data/processed_data/df_1.6.4_计算住院时长.xlsx')
if 'Unnamed: 0' in df_SUP_inp.columns:
    del df_SUP_inp['Unnamed: 0']

In [506]:
df_SUP_inp.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病', '肺部感染', '糖尿病', '高血压', '心功能不全', '脑梗死', '头晕',
       '急性冠脉综合征', '房颤', '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿', '呼吸衰竭', '胸腔积液',
       '尿路感染', '腹膜透析', '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏', '急性肾损伤',
       '心肌梗塞', '心血管疾病_label', '呼吸系统疾病_label', '神经系统疾病_label', '泌尿系统疾病_label',
       '内分泌和代谢疾病_label', '消化系统疾病_label', '危重症疾病_label', '住院日期', '完成日期',
       '住院时长'],
      dtype='object')

### 剔除少于6天的住院时长

In [507]:
df_SUP_inp=df_SUP_inp[df_SUP_inp.住院时长>=6]

In [508]:
df_SUP_inp.shape

(211, 44)

In [54]:
df_SUP_inp.to_csv(project_path+'/data/processed_data/df_1.6.5_剔除少于3天的住院时长.csv')

In [55]:
df_SUP_inp[df_SUP_inp['用药天数']>df_SUP_inp['住院时长']][['用药天数','住院时长']]

,用药天数,住院时长
7,70,23
12,13,10


In [56]:
# ### 出院前3天内用药筛选

# # 统计出院前1天的用药
# df_SUP_oup_1d=df_SUP_inp[(pd.to_datetime(df_SUP_inp['用药结束时间'])>=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=1))&
#     (pd.to_datetime(df_SUP_inp['用药开始时间'])<=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=1))]

# df_SUP_oup_1d.shape

# # 统计出院前2天的用药
# df_SUP_oup_2d=df_SUP_inp[(pd.to_datetime(df_SUP_inp['用药结束时间'])>=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=1))&
#     (pd.to_datetime(df_SUP_inp['用药开始时间'])<=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=2))]

# df_SUP_oup_2d.shape

# # 统计出院前3天的用药
# df_SUP_oup_3d=df_SUP_inp[(pd.to_datetime(df_SUP_inp['用药结束时间'])>=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=1))&
#     (pd.to_datetime(df_SUP_inp['用药开始时间'])<=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=3))]

# df_SUP_oup_3d.shape

# df_SUP_oup_3d.to_excel(project_path+'/data/processed_data/df_1.6.5_出院前3d_用药.xlsx')

# # 统计出院前4天的用药
# df_SUP_oup_4d=df_SUP_inp[(pd.to_datetime(df_SUP_inp['用药结束时间'])>=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=1))&
#     (pd.to_datetime(df_SUP_inp['用药开始时间'])<=pd.to_datetime(df_SUP_inp['完成日期'])-datetime.timedelta(days=4))]

# df_SUP_oup_4d.shape

## 提取人口学特征

In [57]:
df_gender_age_1=pd.read_excel(project_path + '/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',sheet_name='DM_人口学资料',header=1)

df_gender_age_2=pd.read_excel(project_path + '/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',sheet_name='DM_人口学资料',header=1)

df_gender_age_1['受试者编号']=df_gender_age_1['受试者编号'].astype('str')+'_'+'a'
df_gender_age_2['受试者编号']=df_gender_age_2['受试者编号'].astype('str')+'_'+'b'
df_gender_age=pd.concat([df_gender_age_1,df_gender_age_2],axis=0)
df_gender_age=df_gender_age.reset_index(drop=True)

In [58]:
print(df_gender_age_1.shape)
print(df_gender_age_2.shape)

(642, 22)
(104, 22)


In [59]:
# 非空
df_gender_age=df_gender_age[df_gender_age['受试者编号'].notnull()].reset_index(drop=True)

In [60]:
df_gender_age.shape

(746, 22)

In [61]:
df_gender_age.head()

,方案号,中心标识,中心名称,受试者编号,访视,出生日期,年龄,年龄单位,年龄标准值,年龄标准单位,...,身高,身高单位,身高标准值,身高标准单位,体重,体重单位,体重标准值,体重标准单位,原始录入日期,最后修改日期
0,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90001_a,入组访视,1957-02-06,61.0,岁,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/4/20 15:02:45,2019/8/7 22:51:40
1,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90003_a,入组访视,1964-11-23,53.0,岁,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/9/16 15:40:43,2019/8/8 16:42:23
2,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90004_a,入组访视,1948-01-01,70.0,岁,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/10/31 10:59:48,2019/8/7 22:50:39
3,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90005_a,入组访视,1948-01-01,70.0,岁,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/10/31 12:11:00,2019/8/8 16:43:36
4,SMZSY-V1.0,9,上海交通大学医学院附属新华医院,90006_a,入组访视,1939-01-01,79.0,岁,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018/10/31 13:02:01,2019/8/8 18:00:16


In [62]:
df_gender_age=df_gender_age[['受试者编号','年龄','性别_描述']].rename(columns={'性别_描述':'性别'})
df_gender_age['性别']=df_gender_age['性别'].apply(lambda x: 1 if re.match('男',str(x)) else 0)

In [63]:
df_popu=df_gender_age.copy()

In [64]:
df_popu.shape

(746, 3)

In [65]:
df_popu.head()

,受试者编号,年龄,性别
0,90001_a,61.0,0
1,90003_a,53.0,1
2,90004_a,70.0,1
3,90005_a,70.0,1
4,90006_a,79.0,0


In [66]:
df_popu.to_excel(project_path+'/data/processed_data/df_1.7_popu.xlsx')

In [67]:
df_SUP_popu=pd.merge(df_SUP_inp,df_popu,on='受试者编号',how='left')

In [68]:
df_SUP_popu.shape

(211, 46)

In [69]:
df_SUP_popu.to_excel(project_path+'/data/processed_data/df_1.7_df_SUP_popu.xlsx')

## 提取基础疾病

In [70]:
'''
提取个人史(吸烟；饮酒)、既往史(高血压、糖尿病)、家族史、过敏史。
如果个人史既往史不平衡，可以合并成一列：基础疾病。
如果疾病疾病不平衡，则可以删除
'''

'\n提取个人史(吸烟；饮酒)、既往史(高血压、糖尿病)、家族史、过敏史。\n如果个人史既往史不平衡，可以合并成一列：基础疾病。\n如果疾病疾病不平衡，则可以删除\n'

### 个人史既往史

In [71]:
# 其他科 既往史
df_past_history_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='MH_既往史',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
df_past_history_1['受试者编号']=df_past_history_1['受试者编号'].apply(lambda x: str(x)+'_a')
df_past_history_1=df_past_history_1[['受试者编号','既往史']]
# 心内科既往史
df_past_history_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='MH_既往史',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
df_past_history_2['受试者编号']=df_past_history_2['受试者编号'].apply(lambda x: str(x)+'_b')
df_past_history_2=df_past_history_2[['受试者编号','既往史']]

# 合并其他科和心内科--既往史
df_past_history=pd.concat([df_past_history_1,df_past_history_2],axis=0).reset_index(drop=True)

In [72]:
print(df_past_history_1.shape)
print(df_past_history_2.shape)

(640, 2)
(104, 2)


In [73]:
df_past_history.head()

,受试者编号,既往史
0,90001_a,脑梗危险因素：有高血压病，有糖尿病，无高脂血症，无房颤，无脑供血动脉狭窄，无吸烟，无酗酒，无...
1,90003_a,"传染病史:35年患有“乙型脑炎”,病后至今患者反应较迟钝。否认肝炎、结核、伤寒等传染病史。\..."
2,90004_a,传染病史:否认结核伤寒等传染病史。\n药物及其他过敏史:否认药物食物过敏史。\n预防接种史:...
3,90005_a,传染病史:否认肝炎、结核、伤寒等传染病史。\n药物及其他过敏史:否认药物、食物过敏史。\n预...
4,90006_a,"脑梗危险因素:有高血压病等危险因素,无糖尿病,高脂血症,房颤,脑供血动脉狭窄,吸烟,酗..."


In [74]:
df_past_history['吸烟史']=np.nan
df_past_history['饮酒史']=np.nan
df_past_history['肝炎']=np.nan

In [75]:
# 个人史包括吸烟史、饮酒史
# 既往史包括高血压、糖尿病、青霉素过敏史、肝炎、结核史、手术外伤史、输血史、预防接种史

for i in range(df_past_history.shape[0]):
#     if i == 2:
#         break
    record_content=df_past_history.loc[i,'既往史']
    if pd.isnull(record_content):
        continue
    record_list=re.split('，|,|。',record_content)
    # 匹配个人史
    for j in record_list:
        # 匹配吸烟史
        if re.match('[^无否戒没]*烟',j):
            df_past_history.loc[i,'吸烟史']=1
        else:
            df_past_history.loc[i,'吸烟史']= max(0,df_past_history.loc[i,'吸烟史'])
        # 匹配饮酒史
        if re.match('[^无否戒没]*酒[^精]*',j):
            df_past_history.loc[i,'饮酒史']=1
#         elif pd.isnull(df_past_history.loc[i,'饮酒史']):
#             df_past_history.loc[i,'饮酒史']=0
        else:
            df_past_history.loc[i,'饮酒史']=max(0,df_past_history.loc[i,'饮酒史'])
        # 匹配肝炎
        if re.match('[^无否没]*肝炎',j):
            df_past_history.loc[i,'肝炎']=1
        else:
            df_past_history.loc[i,'肝炎']=max(0,df_past_history.loc[i,'肝炎'])


In [76]:
df_past_history=df_past_history.drop(['既往史'],axis=1)
# 增加基础疾病变量

In [77]:
df_past_history.head()

,受试者编号,吸烟史,饮酒史,肝炎
0,90001_a,0.0,0.0,0.0
1,90003_a,0.0,0.0,0.0
2,90004_a,0.0,0.0,0.0
3,90005_a,0.0,0.0,0.0
4,90006_a,1.0,1.0,0.0


In [78]:
df_past_history.to_excel(project_path+'/data/processed_data/基础疾病/df_1.8.1_个人史既往史.xlsx')

### 家族史

In [79]:
# 其他科 家族史
df_family_history_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='FH_家族史',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
df_family_history_1['受试者编号']=df_family_history_1['受试者编号'].apply(lambda x: str(x)+'_a')
df_family_history_1=df_family_history_1[['受试者编号','受试者是否有家族史？_描述']]
# 心内科 家族史
df_family_history_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='FH_家族史',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
df_family_history_2['受试者编号']=df_family_history_2['受试者编号'].apply(lambda x: str(x)+'_b')
df_family_history_2=df_family_history_2[['受试者编号','受试者是否有家族史？_描述']]

# 合并其他科和心内科--既往史
df_family_history=pd.concat([df_family_history_1,df_family_history_2],axis=0).reset_index(drop=True)
df_family_history.rename(columns={'受试者是否有家族史？_描述':'family_history'},inplace=True)

In [80]:
df_family_history.head()

,受试者编号,family_history
0,90001_a,无
1,90003_a,无
2,90004_a,无
3,90005_a,无
4,90006_a,无


In [81]:
df_family_history['family_history']=df_family_history.family_history.apply(lambda x: np.nan if pd.isnull(x) else
                                                                0 if re.match('无',x) else
                                                                1 if re.match('有',x) else x)

In [82]:
df_family_history.head()

,受试者编号,family_history
0,90001_a,0
1,90003_a,0
2,90004_a,0
3,90005_a,0
4,90006_a,0


In [83]:
df_family_history.to_excel(project_path+'/data/processed_data/基础疾病/df_1.8.2_family_history.xlsx')

### 过敏史

In [84]:
# 其他科 过敏史
df_allergy_history_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='AH_过敏史',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
df_allergy_history_1['受试者编号']=df_allergy_history_1['受试者编号'].apply(lambda x: str(x)+'_a')
df_allergy_history_1=df_allergy_history_1[['受试者编号','药物过敏_描述']]
# 心内科 过敏史
df_allergy_history_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='AH_过敏史',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
df_allergy_history_2['受试者编号']=df_allergy_history_2['受试者编号'].apply(lambda x: str(x)+'_b')
df_allergy_history_2=df_allergy_history_2[['受试者编号','药物过敏_描述']]

# 合并其他科和心内科--既往史
df_allergy_history=pd.concat([df_allergy_history_1,df_allergy_history_2],axis=0)
df_allergy_history.rename(columns={'药物过敏_描述':'allergy_history'},inplace=True)

In [85]:
df_allergy_history['allergy_history']=df_allergy_history.allergy_history.apply(lambda x: np.nan if pd.isnull(x) else
                                                                0 if re.match('无',x) else
                                                                1 if re.match('有',x) else x)

In [86]:
df_allergy_history.head()

,受试者编号,allergy_history
0,90001_a,1.0
1,90003_a,0.0
2,90004_a,0.0
3,90005_a,0.0
4,90006_a,0.0


In [87]:
df_allergy_history.to_excel(project_path+'/data/processed_data/基础疾病/df_1.8.3_allergy_history.xlsx')

### 合并既往史、家族史、过敏史

In [88]:
df_history=pd.merge(df_past_history,df_family_history,on='受试者编号',how='outer')
df_history=pd.merge(df_history,df_allergy_history,on='受试者编号',how='outer')

In [89]:
df_history

,受试者编号,吸烟史,饮酒史,肝炎,family_history,allergy_history
0,90001_a,0.0,0.0,0.0,0,1.0
1,90003_a,0.0,0.0,0.0,0,0.0
2,90004_a,0.0,0.0,0.0,0,0.0
3,90005_a,0.0,0.0,0.0,0,0.0
4,90006_a,1.0,1.0,0.0,0,0.0
...,...,...,...,...,...,...
739,90100_b,0.0,0.0,0.0,0,0.0
740,90101_b,0.0,0.0,0.0,0,0.0
741,90102_b,0.0,0.0,0.0,0,0.0
742,90104_b,0.0,0.0,0.0,0,0.0


### 并入个人病史

In [90]:
df_SUP_history=pd.merge(df_SUP_popu,df_history,on='受试者编号',how='left')

In [91]:
df_SUP_history.shape

(211, 51)

In [92]:
df_SUP_history.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病', '肺部感染', '糖尿病', '高血压', '心功能不全', '脑梗死', '头晕',
       '急性冠脉综合征', '房颤', '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿', '呼吸衰竭', '胸腔积液',
       '尿路感染', '腹膜透析', '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏', '急性肾损伤',
       '心肌梗塞', '心血管疾病_label', '呼吸系统疾病_label', '神经系统疾病_label', '泌尿系统疾病_label',
       '内分泌和代谢疾病_label', '消化系统疾病_label', '危重症疾病_label', '住院日期', '完成日期', '住院时长',
       '年龄', '性别', '吸烟史', '饮酒史', '肝炎', 'family_history', 'allergy_history'],
      dtype='object')

### 合并基础疾病

In [93]:
'''
基础疾病：吸烟史,高血压,糖尿病,饮酒史,肝炎
'''

'\n基础疾病：吸烟史,高血压,糖尿病,饮酒史,肝炎\n'

In [94]:
df_SUP_history=df_SUP_history.reset_index(drop=True)
funda_disease=['吸烟史','高血压','糖尿病','饮酒史','肝炎']
for i in range(df_SUP_history.shape[0]):
    # 增加基础疾病变量
    if df_SUP_history.loc[i,funda_disease].sum() >=1:
        df_SUP_history.loc[i,'基础疾病']=1
    else:
        df_SUP_history.loc[i,'基础疾病']=0

In [95]:
df_SUP_history=df_SUP_history.drop(funda_disease,axis=1)

In [96]:
df_SUP_history.shape

(211, 47)

In [97]:
df_SUP_history.to_excel(project_path+'/data/processed_data/df_1.8.6_并入基础疾病.xlsx')

## 提取联合用药

### 提取联合用药

In [98]:
# 其他科
df_drug_other_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='CM_合并用药',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

# 标记受试者编号
df_drug_other_1['受试者编号']=df_drug_other_1['受试者编号'].apply(lambda x: str(x)+'_a')

# 提取有效字段
df_drug_other_1=df_drug_other_1[['受试者编号','药物名称（通用名）','用药开始日期','用药结束日期']]

In [99]:
df_drug_other_1

,受试者编号,药物名称（通用名）,用药开始日期,用药结束日期
0,90001_a,乳果糖口服液,2018-04-08,2018-04-15
1,90001_a,依达拉奉注射液,2018-04-09,2018-04-15
2,90001_a,精蛋白生物合成人胰岛素30R,2018-04-11,2018-04-15
3,90001_a,二甲双胍片,2018-04-14,2018-04-15
4,90001_a,醒脑静注射液,2018-04-14,2018-04-15
...,...,...,...,...
1687,90645_a,曲美他嗪片,2019-08-08,2019-08-17
1688,90645_a,银杏内脂注射液,2019-08-08,2019-08-17
1689,90646_a,巴氯芬片,2019-08-20,2019-08-22
1690,90646_a,泮托拉唑肠溶片,2019-08-20,2019-08-22


In [100]:
# 心内科
df_drug_other_2 = pd.read_excel(project_path+'/data/raw_data//导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='CM_合并用药',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

# 标记受试者编号
df_drug_other_2['受试者编号']=df_drug_other_2['受试者编号'].apply(lambda x: str(x)+'_b')
# 提取有效字段
df_drug_other_2=df_drug_other_2[['受试者编号','药物名称（通用名）','用药开始日期','用药结束日期']]

In [101]:
print(df_drug_other_1.shape)
print(df_drug_other_2.shape)

(1692, 4)
(311, 4)


In [102]:
# 合并其他科和心内科联合用药
df_drug_other=pd.concat([df_drug_other_1,df_drug_other_2])

In [103]:
df_drug_other['药物名称（通用名）'].nunique()

341

In [104]:
df_drug_other.head()

,受试者编号,药物名称（通用名）,用药开始日期,用药结束日期
0,90001_a,乳果糖口服液,2018-04-08,2018-04-15
1,90001_a,依达拉奉注射液,2018-04-09,2018-04-15
2,90001_a,精蛋白生物合成人胰岛素30R,2018-04-11,2018-04-15
3,90001_a,二甲双胍片,2018-04-14,2018-04-15
4,90001_a,醒脑静注射液,2018-04-14,2018-04-15


In [105]:
df_drug_other.to_excel(project_path+'/data/processed_data/df_1.9.1_联合用药.xlsx')

### 冠心病和心梗患者联合用药

In [106]:
df_drug_other_choose=df_drug_other[df_drug_other.受试者编号.isin(df_SUP_history.受试者编号.unique())]

In [107]:
df_drug_other_choose.shape

(599, 4)

In [108]:
df_drug_other_choose.to_excel(project_path+'/data/processed_data/df_1.9.2_冠心病和心梗患者联合用药.xlsx')

### 筛选出院前3天的联合用药

In [109]:
# # 提取出院前3天的联合用药
# temp_list=[]
# for i in df_SUP_history.受试者编号.unique():
#     temp=df_SUP_history[df_SUP_history['受试者编号']==i]
#     temp=temp.reset_index(drop=True)
#     temp_other=df_drug_other[df_drug_other['受试者编号']==i]
#     temp_other=temp_other.reset_index(drop=True)
    
#     temp_other_choose=temp_other[(pd.to_datetime(temp_other['用药结束日期'])>=pd.to_datetime(temp.loc[0,'完成日期'])-datetime.timedelta(days=1))&
#                                  (pd.to_datetime(temp_other['用药开始日期'])<=pd.to_datetime(temp.loc[0,'完成日期'])-datetime.timedelta(days=3))]

#     temp_list.append(temp_other_choose)

In [110]:
# df_drug_other_choose=pd.concat(temp_list,axis=0).reset_index(drop=True)
# del temp_list

In [111]:
df_drug_other_choose=df_drug_other_choose.rename(columns={'药物名称（通用名）':'drug_name'})

In [112]:
df_drug_other_choose.head()

,受试者编号,drug_name,用药开始日期,用药结束日期
28,90022_a,莫西沙星氯化钠针,2018-02-23,2018-02-23
63,90028_a,0.9%氯化钠针,2018-02-24,2018-03-03
64,90028_a,谷胱甘肽针,2018-02-24,2018-03-03
65,90028_a,硝酸异山梨酯针,2018-02-24,2018-03-03
66,90028_a,托拉塞米针,2018-02-24,2018-03-03


In [113]:
# df_drug_other_choose.to_excel(project_path+'/data/processed_data/df_1.9.2_出院前3天内的联合用药.xlsx')

### 统计联合用药

In [114]:
drug_name_counts=df_drug_other_choose['drug_name'].value_counts()
df_drug_other_name=pd.DataFrame(data={'药物名称':drug_name_counts.index,
                                     '频次':drug_name_counts.values})
# 保存联合用药名
df_drug_other_name.to_excel(project_path+'/data/processed_data/df_1.9.3_统计联合用药名.xlsx')

### 选取重要联合用药

In [115]:
'''
1、降血脂药：
（1）HMG-CoA还原酶抑制剂（他汀类）：阿托伐他汀钙片、瑞舒伐他汀钙片、阿伐他汀钙片（系书写错误、正确为：阿托伐他汀钙片）、啊伐他丁钙片（系书写错误、正确为：阿托伐他汀钙片）
（2）烟酸类：烟酸注射液
2、抗血小板药物：阿司匹灵肠溶片、阿司匹林肠溶片（所有阿司匹林为一类）、替格瑞洛片 倍林达、硫酸氢氯吡格雷片、替格瑞溶片培林达、替格瑞洛片、阿司匹林溶肠片、替格瑞洛片 倍达林（把所有替格瑞洛当成一种药）、替格瑞洛片 培林达、硫酸氢氯吡格雷片（所有氯吡格雷为一类）、硫酸氢氯吡克雷特片、硫酸氨氯吡格雷片（系书写错误、正确为：硫酸氢氯吡格雷片）
3、抗凝药物：达肝素钠注射液、纤溶酶针、达肝索钠注射液（系书写错误、正确为：达肝素钠注射液）、依诺肝素钠针、达肝素钠主射液（系书写错误、正确为：达肝素钠注射液）、华法林钠片、依诺肝素钠针/克赛
'''

'\n1、降血脂药：\n（1）HMG-CoA还原酶抑制剂（他汀类）：阿托伐他汀钙片、瑞舒伐他汀钙片、阿伐他汀钙片（系书写错误、正确为：阿托伐他汀钙片）、啊伐他丁钙片（系书写错误、正确为：阿托伐他汀钙片）\n（2）烟酸类：烟酸注射液\n2、抗血小板药物：阿司匹灵肠溶片、阿司匹林肠溶片（所有阿司匹林为一类）、替格瑞洛片 倍林达、硫酸氢氯吡格雷片、替格瑞溶片培林达、替格瑞洛片、阿司匹林溶肠片、替格瑞洛片 倍达林（把所有替格瑞洛当成一种药）、替格瑞洛片 培林达、硫酸氢氯吡格雷片（所有氯吡格雷为一类）、硫酸氢氯吡克雷特片、硫酸氨氯吡格雷片（系书写错误、正确为：硫酸氢氯吡格雷片）\n3、抗凝药物：达肝素钠注射液、纤溶酶针、达肝索钠注射液（系书写错误、正确为：达肝素钠注射液）、依诺肝素钠针、达肝素钠主射液（系书写错误、正确为：达肝素钠注射液）、华法林钠片、依诺肝素钠针/克赛\n'

In [116]:
df_drug_降血脂药=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('阿托伐他汀钙片|瑞舒伐他汀钙片|阿伐他汀钙片|啊伐他丁钙片|烟酸注射液')]
df_drug_抗血小板药=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('阿司匹灵肠溶片|阿司匹林|替格瑞洛片|硫酸氢氯吡格雷片|替格瑞溶片培林达|硫酸氢氯吡克雷特片|硫酸氨氯吡格雷片')]
df_drug_抗凝药=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('达肝素钠注射液|纤溶酶针|达肝索钠注射液|依诺肝素钠针|达肝素钠主射液|华法林钠片')]

In [117]:
df_drug_降血脂药.drug_name.unique()

array(['阿托伐他汀钙片', '瑞舒伐他汀钙片', '阿伐他汀钙片', '烟酸注射液'], dtype=object)

In [118]:
df_drug_降血脂药.to_excel(project_path+'/data/processed_data/联合用药/df_1.9.4_降血脂药.xlsx')
df_drug_抗血小板药.to_excel(project_path+'/data/processed_data/联合用药/df_1.9.4_抗血小板药.xlsx')
df_drug_抗凝药.to_excel(project_path+'/data/processed_data/联合用药/df_1.9.4_抗凝药.xlsx')

### 合并联合用药

In [119]:
df_SUP_drug_other=df_SUP_history.copy()

In [120]:
df_SUP_drug_other.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病', '肺部感染', '心功能不全', '脑梗死', '头晕', '急性冠脉综合征', '房颤',
       '肾功能不全', '尿毒症', '脓毒血症', '肺水肿', '肺气肿', '呼吸衰竭', '胸腔积液', '尿路感染', '腹膜透析',
       '肝功能不全', '高脂血症', '慢性支气管炎', '胆囊结石', '早搏', '急性肾损伤', '心肌梗塞', '心血管疾病_label',
       '呼吸系统疾病_label', '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label',
       '消化系统疾病_label', '危重症疾病_label', '住院日期', '完成日期', '住院时长', '年龄', '性别',
       'family_history', 'allergy_history', '基础疾病'],
      dtype='object')

In [121]:
# 并入联合用药：降脂药、降血小板药、抗凝药
df_SUP_drug_other['降脂药']=df_SUP_drug_other.受试者编号.apply(lambda x: 1 if x in df_drug_降血脂药.受试者编号.unique() else 0)
df_SUP_drug_other['降血小板药']=df_SUP_drug_other.受试者编号.apply(lambda x: 1 if x in df_drug_抗血小板药.受试者编号.unique() else 0)
df_SUP_drug_other['抗凝药']=df_SUP_drug_other.受试者编号.apply(lambda x: 1 if x in df_drug_抗凝药.受试者编号.unique() else 0)

In [122]:
df_SUP_drug_other.shape

(211, 50)

In [123]:
df_SUP_drug_other.to_excel(project_path+'/data/processed_data/df_1.9.5_合并联合用药.xlsx')

In [124]:
# ## 提取影像检查
# # 其他科-影像检查
# df_影像_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
#                             sheet_name='ID_影像检查',
#                            header=1,)  # 指定第2行作为标题行
# #                            skiprows=[1,1,1])  # 跳过第1行
# # 心内科-影像检查
# df_影像_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
#                             sheet_name='ID_影像检查',
#                            header=1,)  # 指定第2行作为标题行
# #                            skiprows=[1,1,1])  # 跳过第几行)

# # 标记受试者编号
# df_影像_1['受试者编号']=df_影像_1['受试者编号'].apply(lambda x: str(x)+'_a')
# # 标记受试者编号
# df_影像_2['受试者编号']=df_影像_2['受试者编号'].apply(lambda x: str(x)+'_b')

# print(df_影像_1.shape)
# print(df_影像_2.shape)

# df_影像_1.columns

# # 提取有效字段
# df_影像_1=df_影像_1[['受试者编号','检查日期', '检查项目_描述','检查结果']]
# # 提取有效字段
# df_影像_2=df_影像_2[['受试者编号','检查日期', '检查项目_描述','检查结果']]
# # 合并影像检查
# df_影像 = pd.concat([df_影像_1,df_影像_2],axis=0)
# # 重复检查取最后一条
# df_影像 = df_影像.sort_values(['受试者编号','检查日期'],ascending=True).drop_duplicates(['受试者编号','检查项目_描述'],keep='last')
# # 重命名
# df_影像=df_影像.rename(columns={'检查日期':'影像检查日期',
#                                '检查项目_描述':'影像检查项目',
#                                '检查结果':'影像检查结果'})

# df_影像.head()

# df_影像.head()

# # 并入出入院-血常规检查
# df_SUP_image=pd.merge(df_SUP_drug_other,df_影像,on='受试者编号',how='left')

# df_SUP_image.shape

# df_SUP_image.head()

# df_SUP_image.to_excel(project_path+'/data/processed_data/df_1.10_合并影像检查.xlsx')

## 提取检测信息

In [125]:
df_SUP_test=df_SUP_drug_other.copy()

### 血常规

In [126]:
# 其他科-入院血常规
df_血常规_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='LBR_血常规',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
# 心内科-入院血常规
df_血常规_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='LBR_血常规',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)

In [127]:
# 标记受试者编号
df_血常规_1['受试者编号']=df_血常规_1['受试者编号'].apply(lambda x: str(x)+'_a')
# 标记受试者编号
df_血常规_2['受试者编号']=df_血常规_2['受试者编号'].apply(lambda x: str(x)+'_b')

In [128]:
df_血常规_1.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', '红细胞', '红细胞-临床意义',
       '红细胞-临床意义_描述', '红细胞-异常请描述', '血红蛋白', '血红蛋白-临床意义', '血红蛋白-临床意义_描述',
       '血红蛋白-异常请描述', '白细胞', '白细胞-临床意义', '白细胞-临床意义_描述', '白细胞-异常请描述', '中性粒细胞计数',
       '中性粒细胞计数-临床意义', '中性粒细胞计数-临床意义_描述', '中性粒细胞计数-异常请描述', '血小板', '血小板-临床意义',
       '血小板-临床意义_描述', '血小板-异常请描述', '红细胞压积', '红细胞压积-临床意义', '红细胞压积-临床意义_描述',
       '红细胞压积-异常请描述', '原始录入日期', '最后修改日期'],
      dtype='object')

In [129]:
df_血常规_2.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', '红细胞', '红细胞-临床意义',
       '红细胞-临床意义_描述', '红细胞-异常请描述', '血红蛋白', '血红蛋白-临床意义', '血红蛋白-临床意义_描述',
       '血红蛋白-异常请描述', '白细胞', '白细胞-临床意义', '白细胞-临床意义_描述', '白细胞-异常请描述', '中性粒细胞计数',
       '中性粒细胞计数-临床意义', '中性粒细胞计数-临床意义_描述', '中性粒细胞计数-异常请描述', '血小板', '血小板-临床意义',
       '血小板-临床意义_描述', '血小板-异常请描述', '红细胞压积', '红细胞压积-临床意义', '红细胞压积-临床意义_描述',
       '红细胞压积-异常请描述', '原始录入日期', '最后修改日期'],
      dtype='object')

In [130]:
# 提取有效字段
df_血常规_1=df_血常规_1[['访视','受试者编号','检查日期','红细胞','血红蛋白','白细胞','中性粒细胞计数','血小板','红细胞压积']]
# 提取有效字段
df_血常规_2=df_血常规_2[['访视','受试者编号','检查日期','红细胞','血红蛋白','白细胞','中性粒细胞计数','血小板','红细胞压积']]

In [131]:
df_血常规_1.shape

(1176, 9)

In [132]:
df_血常规_1.head()

,访视,受试者编号,检查日期,红细胞,血红蛋白,白细胞,中性粒细胞计数,血小板,红细胞压积
0,入院检查,90001_a,2018-03-31,4.38,127.0,10.6,75.4,315.0,37.2
1,出院前检查,90001_a,2018-04-10,3.85,111.0,15.0,75.0,337.0,32.5
2,入院检查,90003_a,2018-01-31,4.70,133.0,1.8,87.5,136.0,38.7
3,出院前检查,90003_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,入院检查,90004_a,2018-01-26,2.99,97.0,6.2,59.9,226.0,29.0


#### 入院检查

In [133]:
# 其他科--入院血常规检测
df_血常规_1_inp=df_血常规_1[df_血常规_1['访视'].str.contains('入院检查')]
df_血常规_1_inp=df_血常规_1_inp.reset_index(drop=True)
# 心内科--入院血常规检测
df_血常规_2_inp=df_血常规_2[df_血常规_2['访视'].str.contains('入院检查')]
df_血常规_2_inp=df_血常规_2_inp.reset_index(drop=True)
# 合并入院血常规检查
df_血常规_inp = pd.concat([df_血常规_1_inp,df_血常规_2_inp],axis=0)

In [134]:
df_血常规_inp.shape

(745, 9)

In [135]:
df_血常规_inp.head()

,访视,受试者编号,检查日期,红细胞,血红蛋白,白细胞,中性粒细胞计数,血小板,红细胞压积
0,入院检查,90001_a,2018-03-31,4.38,127.0,10.60,75.4,315.0,37.2
1,入院检查,90003_a,2018-01-31,4.70,133.0,1.80,87.5,136.0,38.7
2,入院检查,90004_a,2018-01-26,2.99,97.0,6.20,59.9,226.0,29.0
3,入院检查,90005_a,2018-02-08,3.26,104.0,14.21,80.5,243.0,30.5
4,入院检查,90006_a,2018-02-12,4.36,125.0,8.00,58.5,260.0,37.7


In [136]:
df_血常规_inp.to_excel(project_path+'/data/processed_data/检测数据/df_血常规_入院检查.xlsx')

#### 出院检查

In [137]:
# 其他科--出院血常规检测
df_血常规_1_oup=df_血常规_1[df_血常规_1['访视'].str.contains('出院前检查')]
df_血常规_1_oup=df_血常规_1_oup.reset_index(drop=True)
# 心内科--出院血常规检测
df_血常规_2_oup=df_血常规_2[df_血常规_2['访视'].str.contains('出院前检查')]
df_血常规_2_oup=df_血常规_2_oup.reset_index(drop=True)
# 合并出院院血常规检查
df_血常规_oup = pd.concat([df_血常规_1_oup,df_血常规_2_oup],axis=0)

In [138]:
df_血常规_oup.shape

(630, 9)

In [139]:
df_血常规_oup.head()

,访视,受试者编号,检查日期,红细胞,血红蛋白,白细胞,中性粒细胞计数,血小板,红细胞压积
0,出院前检查,90001_a,2018-04-10,3.85,111.0,15.0,75.0,337.0,32.5
1,出院前检查,90003_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,出院前检查,90004_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,出院前检查,90005_a,2018-02-13,3.11,103.0,13.3,67.7,194.0,29.1
4,出院前检查,90006_a,2018-02-19,4.10,120.0,9.6,73.1,256.0,35.2


In [140]:
df_血常规_oup.to_excel(project_path+'/data/processed_data/检测数据/df_血常规_出院检查.xlsx')

#### 修改列名

In [141]:
df_血常规_inp=df_血常规_inp.drop(['访视'],axis=1)
df_血常规_oup=df_血常规_oup.drop(['访视'],axis=1)

In [142]:
for i in df_血常规_inp.columns[1:]:
    df_血常规_inp.rename(columns={i:i + '_入院'},inplace=True)
for i in df_血常规_oup.columns[1:]:
    df_血常规_oup.rename(columns={i:i+'_出院'},inplace=True)  

In [143]:
df_血常规_inp.columns

Index(['受试者编号', '检查日期_入院', '红细胞_入院', '血红蛋白_入院', '白细胞_入院', '中性粒细胞计数_入院',
       '血小板_入院', '红细胞压积_入院'],
      dtype='object')

In [144]:
df_test_血常规=pd.merge(df_血常规_inp,df_血常规_oup,on='受试者编号',how='outer')

In [145]:
df_test_血常规.head()

,受试者编号,检查日期_入院,红细胞_入院,血红蛋白_入院,白细胞_入院,中性粒细胞计数_入院,血小板_入院,红细胞压积_入院,检查日期_出院,红细胞_出院,血红蛋白_出院,白细胞_出院,中性粒细胞计数_出院,血小板_出院,红细胞压积_出院
0,90001_a,2018-03-31,4.38,127.0,10.60,75.4,315.0,37.2,2018-04-10,3.85,111.0,15.0,75.0,337.0,32.5
1,90003_a,2018-01-31,4.70,133.0,1.80,87.5,136.0,38.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90004_a,2018-01-26,2.99,97.0,6.20,59.9,226.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90005_a,2018-02-08,3.26,104.0,14.21,80.5,243.0,30.5,2018-02-13,3.11,103.0,13.3,67.7,194.0,29.1
4,90006_a,2018-02-12,4.36,125.0,8.00,58.5,260.0,37.7,2018-02-19,4.10,120.0,9.6,73.1,256.0,35.2


In [146]:
df_test_血常规.to_excel(project_path+'/data/processed_data/df_1.10.1.3_检查_血常规.xlsx')

#### 统计血常规-住院天数

In [147]:
# df_test_血常规=df_test_血常规[['受试者编号','检查日期_入院']]
# df_inp_oup_time=df_SUP_history[['受试者编号','住院日期', '完成日期']]

# # 合并检测时间和住院时间
# df_血常规_time=pd.merge(df_血常规_入院_time,df_血常规_出院_time,on='受试者编号',how='right')
# df_血常规_inp_oup = pd.merge(df_血常规_time,df_inp_oup_time,on='受试者编号',how='inner')

# df_血常规_inp_oup['血常规-入院']=pd.to_datetime(df_血常规_inp_oup['住院日期'])-pd.to_datetime(df_血常规_inp_oup['检查日期_入院'])
# df_血常规_inp_oup['血常规-入院']=df_血常规_inp_oup['血常规-入院'].apply(lambda x:x.days+1)

# df_血常规_inp_oup['血常规-出院']=pd.to_datetime(df_血常规_inp_oup['完成日期'])-pd.to_datetime(df_血常规_inp_oup['检查日期_出院'])
# df_血常规_inp_oup['血常规-出院']=df_血常规_inp_oup['血常规-出院'].apply(lambda x:x.days+1)

# df_血常规_inp_oup.to_excel(project_path+'/data/processed_data/df_1.9.1.4_血常规-出入院时间统计.xlsx')

#### 并入血常规检查

In [148]:
df_test_血常规.head()

,受试者编号,检查日期_入院,红细胞_入院,血红蛋白_入院,白细胞_入院,中性粒细胞计数_入院,血小板_入院,红细胞压积_入院,检查日期_出院,红细胞_出院,血红蛋白_出院,白细胞_出院,中性粒细胞计数_出院,血小板_出院,红细胞压积_出院
0,90001_a,2018-03-31,4.38,127.0,10.60,75.4,315.0,37.2,2018-04-10,3.85,111.0,15.0,75.0,337.0,32.5
1,90003_a,2018-01-31,4.70,133.0,1.80,87.5,136.0,38.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90004_a,2018-01-26,2.99,97.0,6.20,59.9,226.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90005_a,2018-02-08,3.26,104.0,14.21,80.5,243.0,30.5,2018-02-13,3.11,103.0,13.3,67.7,194.0,29.1
4,90006_a,2018-02-12,4.36,125.0,8.00,58.5,260.0,37.7,2018-02-19,4.10,120.0,9.6,73.1,256.0,35.2


In [149]:
# 删除检查日期
df_test_血常规=df_test_血常规.drop(['检查日期_入院','检查日期_出院'],axis=1)
# 并入出入院-血常规检查
df_SUP_test=pd.merge(df_SUP_test,df_test_血常规,on='受试者编号',how='left')

In [150]:
df_SUP_test.head()

,受试者编号,出院诊断,主要疾病转归_描述,用药开始时间,用药结束时间,用药天数,EXDOSFRQ,EXDSTXT,日剂量,冠心病,...,白细胞_入院,中性粒细胞计数_入院,血小板_入院,红细胞压积_入院,红细胞_出院,血红蛋白_出院,白细胞_出院,中性粒细胞计数_出院,血小板_出院,红细胞压积_出院
0,90002_b,"急性前壁心肌梗塞,冠心病killip Ⅰ级,急诊LAD-PCI术; 2型糖尿病",0,2019-04-24,2019-05-05,12,1,60,60,1,...,9.84,NaN,189.0,0.21,NaN,NaN,NaN,NaN,NaN,NaN
1,90006_b,":急性前壁心肌梗塞,killip I级,冠心病 急诊PCI术后 高血压病(2级,极高危),类...",0,2019-04-17,2019-04-24,8,1,60,60,1,...,10.20,90.4,282.0,31.20,NaN,NaN,NaN,NaN,NaN,NaN
2,90007_b,":急性下壁心肌梗塞,Killip I级,急诊RCA-PCI术后;冠心病 三支病变;2型糖尿病...",0,2019-05-20,2019-05-21,2,1,60,60,1,...,9.83,85.1,209.0,39.50,NaN,NaN,NaN,NaN,NaN,NaN
3,90008_b,"急性下壁心肌梗塞,冠心病 ,killip Ⅰ级,急诊PCI术后,甲状腺功能亢进,胆囊结石。",0,2019-06-26,2019-06-27,2,1,60,60,1,...,9.82,69.8,267.0,44.40,NaN,NaN,NaN,NaN,NaN,NaN
4,90009_b,"急性前壁心肌梗塞,killip Ⅳ级,急诊PCI术后 冠心病 心功能不全 高血压病1级,很高...",0,2019-04-25,2019-04-27,3,1,60,60,1,...,20.80,83.5,267.0,44.10,NaN,NaN,NaN,NaN,NaN,NaN


### 尿常规

In [151]:
# 其他科-尿常规检查
df_尿常规_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='LBU_尿常规',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
# 标记受试者编号
df_尿常规_1['受试者编号']=df_尿常规_1['受试者编号'].apply(lambda x: str(x)+'_a')
# 心内科-尿常规检查
df_尿常规_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='LBU_尿常规',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
# 标记受试者编号
df_尿常规_2['受试者编号']=df_尿常规_2['受试者编号'].apply(lambda x: str(x)+'_b')

In [152]:
df_尿常规_1.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', '尿蛋白（PRO）',
       '尿蛋白（PRO）-临床意义', '尿蛋白（PRO）-临床意义_描述', '尿蛋白（PRO）-异常请描述', '尿白细胞（LEU）',
       '尿白细胞（LEU）-临床意义', '尿白细胞（LEU）-临床意义_描述', '尿白细胞（LEU）-异常请描述', '尿红细胞（ERY）',
       '尿红细胞（ERY）-临床意义', '尿红细胞（ERY）-临床意义_描述', '尿红细胞（ERY）-异常请描述', '原始录入日期',
       '最后修改日期'],
      dtype='object')

In [153]:
df_尿常规_2.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', '尿蛋白（PRO）',
       '尿蛋白（PRO）-临床意义', '尿蛋白（PRO）-临床意义_描述', '尿蛋白（PRO）-异常请描述', '尿白细胞（LEU）',
       '尿白细胞（LEU）-临床意义', '尿白细胞（LEU）-临床意义_描述', '尿白细胞（LEU）-异常请描述', '尿红细胞（ERY）',
       '尿红细胞（ERY）-临床意义', '尿红细胞（ERY）-临床意义_描述', '尿红细胞（ERY）-异常请描述', '原始录入日期',
       '最后修改日期'],
      dtype='object')

In [154]:
# 提取有效字段
df_尿常规_1=df_尿常规_1[['访视','受试者编号','检查日期','尿蛋白（PRO）','尿白细胞（LEU）','尿红细胞（ERY）']]
df_尿常规_2=df_尿常规_2[['访视','受试者编号','检查日期','尿蛋白（PRO）','尿白细胞（LEU）','尿红细胞（ERY）']]

In [155]:
df_尿常规_1.shape

(1167, 6)

In [156]:
df_尿常规_1.head()

,访视,受试者编号,检查日期,尿蛋白（PRO）,尿白细胞（LEU）,尿红细胞（ERY）
0,入院检查,90001_a,2018-03-31,25,5-8,0
1,出院前检查,90001_a,NaN,NaN,NaN,NaN
2,入院检查,90003_a,NaN,NaN,NaN,NaN
3,出院前检查,90003_a,NaN,NaN,NaN,NaN
4,入院检查,90004_a,NaN,NaN,NaN,NaN


#### 入院检查

In [157]:
# 入院尿常规检测
df_尿常规_1_inp=df_尿常规_1[df_尿常规_1['访视'].str.contains('入院检查')]
df_尿常规_1_inp=df_尿常规_1_inp.reset_index(drop=True)
df_尿常规_2_inp=df_尿常规_2[df_尿常规_2['访视'].str.contains('入院检查')]
df_尿常规_2_inp=df_尿常规_2_inp.reset_index(drop=True)
# 合并入院尿常规检查
df_尿常规_inp=pd.concat([df_尿常规_1_inp,df_尿常规_2_inp],axis=0)

In [158]:
df_尿常规_inp.shape

(738, 6)

In [159]:
df_尿常规_inp.head()

,访视,受试者编号,检查日期,尿蛋白（PRO）,尿白细胞（LEU）,尿红细胞（ERY）
0,入院检查,90001_a,2018-03-31,25,5-8,0
1,入院检查,90003_a,NaN,NaN,NaN,NaN
2,入院检查,90004_a,NaN,NaN,NaN,NaN
3,入院检查,90005_a,NaN,NaN,NaN,NaN
4,入院检查,90006_a,2018-02-12,阴性,阴性,10


In [160]:
df_尿常规_inp.to_excel(project_path+'/data/processed_data/检测数据/df_尿常规_入院检查.xlsx')

#### 出院检查

In [161]:
# 出院尿常规检查
df_尿常规_1_oup=df_尿常规_1[df_尿常规_1['访视'].str.contains('出院前检查')]
df_尿常规_1_oup=df_尿常规_1_oup.reset_index(drop=True)
df_尿常规_2_oup=df_尿常规_2[df_尿常规_2['访视'].str.contains('出院前检查')]
df_尿常规_2_oup=df_尿常规_2_oup.reset_index(drop=True)
# 合并尿常规检查
df_尿常规_oup=pd.concat([df_尿常规_1_oup,df_尿常规_2_oup],axis=0)

In [162]:
df_尿常规_oup.shape

(617, 6)

In [163]:
df_尿常规_oup.head()

,访视,受试者编号,检查日期,尿蛋白（PRO）,尿白细胞（LEU）,尿红细胞（ERY）
0,出院前检查,90001_a,NaN,NaN,NaN,NaN
1,出院前检查,90003_a,NaN,NaN,NaN,NaN
2,出院前检查,90004_a,NaN,NaN,NaN,NaN
3,出院前检查,90005_a,NaN,NaN,NaN,NaN
4,出院前检查,90006_a,NaN,NaN,NaN,NaN


In [164]:
df_尿常规_oup.to_excel(project_path+'/data/processed_data/检测数据/df_尿常规_出院检查.xlsx')

#### 修改列名

In [165]:
# 删除访视
df_尿常规_inp=df_尿常规_inp.drop(['访视'],axis=1)
df_尿常规_oup=df_尿常规_oup.drop(['访视'],axis=1)
# 修改列名
for i in df_尿常规_inp.columns[1:]:
    df_尿常规_inp.rename(columns={i:i + '_入院'},inplace=True)
for i in df_尿常规_oup.columns[1:]:
    df_尿常规_oup.rename(columns={i:i+'_出院'},inplace=True)  

In [166]:
df_test_尿常规=pd.merge(df_尿常规_inp,df_尿常规_oup,on='受试者编号',how='outer')

In [167]:
df_test_尿常规.head()

,受试者编号,检查日期_入院,尿蛋白（PRO）_入院,尿白细胞（LEU）_入院,尿红细胞（ERY）_入院,检查日期_出院,尿蛋白（PRO）_出院,尿白细胞（LEU）_出院,尿红细胞（ERY）_出院
0,90001_a,2018-03-31,25,5-8,0,NaN,NaN,NaN,NaN
1,90003_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90004_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90005_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90006_a,2018-02-12,阴性,阴性,10,NaN,NaN,NaN,NaN


In [168]:
df_test_尿常规.to_excel(project_path+'/data/processed_data/df_1.10.2.3_尿常规_检查.xlsx')

#### 并入尿常规检查

In [169]:
df_test_尿常规.head()

,受试者编号,检查日期_入院,尿蛋白（PRO）_入院,尿白细胞（LEU）_入院,尿红细胞（ERY）_入院,检查日期_出院,尿蛋白（PRO）_出院,尿白细胞（LEU）_出院,尿红细胞（ERY）_出院
0,90001_a,2018-03-31,25,5-8,0,NaN,NaN,NaN,NaN
1,90003_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90004_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90005_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90006_a,2018-02-12,阴性,阴性,10,NaN,NaN,NaN,NaN


In [170]:
# 删除检查日期
df_test_尿常规=df_test_尿常规.drop(['检查日期_入院','检查日期_出院'],axis=1)
# 并入出入院-血常规检查
df_SUP_test=pd.merge(df_SUP_test,df_test_尿常规,on='受试者编号',how='left')

In [171]:
df_SUP_test.head()

,受试者编号,出院诊断,主要疾病转归_描述,用药开始时间,用药结束时间,用药天数,EXDOSFRQ,EXDSTXT,日剂量,冠心病,...,白细胞_出院,中性粒细胞计数_出院,血小板_出院,红细胞压积_出院,尿蛋白（PRO）_入院,尿白细胞（LEU）_入院,尿红细胞（ERY）_入院,尿蛋白（PRO）_出院,尿白细胞（LEU）_出院,尿红细胞（ERY）_出院
0,90002_b,"急性前壁心肌梗塞,冠心病killip Ⅰ级,急诊LAD-PCI术; 2型糖尿病",0,2019-04-24,2019-05-05,12,1,60,60,1,...,NaN,NaN,NaN,NaN,75,阴性,25,NaN,NaN,NaN
1,90006_b,":急性前壁心肌梗塞,killip I级,冠心病 急诊PCI术后 高血压病(2级,极高危),类...",0,2019-04-17,2019-04-24,8,1,60,60,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90007_b,":急性下壁心肌梗塞,Killip I级,急诊RCA-PCI术后;冠心病 三支病变;2型糖尿病...",0,2019-05-20,2019-05-21,2,1,60,60,1,...,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN
3,90008_b,"急性下壁心肌梗塞,冠心病 ,killip Ⅰ级,急诊PCI术后,甲状腺功能亢进,胆囊结石。",0,2019-06-26,2019-06-27,2,1,60,60,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90009_b,"急性前壁心肌梗塞,killip Ⅳ级,急诊PCI术后 冠心病 心功能不全 高血压病1级,很高...",0,2019-04-25,2019-04-27,3,1,60,60,1,...,NaN,NaN,NaN,NaN,NaN,0.2,2.4,NaN,NaN,NaN


### 血生化

In [172]:
# 其他科-血生化检查
df_血生化_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='LBB_血生化',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
# 标记受试者编号
df_血生化_1['受试者编号']=df_血生化_1['受试者编号'].apply(lambda x: str(x)+'_a')
# 心内科-血生化检查
df_血生化_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='LBB_血生化',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
# 标记受试者编号
df_血生化_2['受试者编号']=df_血生化_2['受试者编号'].apply(lambda x: str(x)+'_b')

In [173]:
df_血生化_1.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', '总胆红素（TBil）',
       '总胆红素（TBil）-临床意义', '总胆红素（TBil）-临床意义_描述', '总胆红素（TBil）-异常请描述',
       '直接胆红素（DBil）', '直接胆红素（DBil）-临床意义', '直接胆红素（DBil）-临床意义_描述',
       '直接胆红素（DBil）-异常请描述', '白蛋白', '白蛋白-临床意义', '白蛋白-临床意义_描述', '白蛋白-异常请描述',
       '前白蛋白', '前白蛋白-临床意义', '前白蛋白-临床意义_描述', '前白蛋白-异常请描述', 'ALT', 'ALT-临床意义',
       'ALT-临床意义_描述', 'ALT-异常请描述', 'AST', 'AST-临床意义', 'AST-临床意义_描述',
       'AST-异常请描述', 'GGT', 'GGT-临床意义', 'GGT-临床意义_描述', 'GGT-异常请描述', 'AKP',
       'AKP-临床意义', 'AKP-临床意义_描述', 'AKP-异常请描述', '尿素氮（BUN）', '尿素氮（BUN）-临床意义',
       '尿素氮（BUN）-临床意义_描述', '尿素氮（BUN）-异常请描述', '肌酐（Cr）', '肌酐（Cr）-临床意义',
       '肌酐（Cr）-临床意义_描述', '肌酐（Cr）-异常请描述', 'PT', 'PT-临床意义', 'PT-临床意义_描述',
       'PT-异常请描述', 'INR', 'INR-临床意义', 'INR-临床意义_描述', 'INR-异常请描述', '血钠',
       '血钠-临床意义', '血钠-临床意义_描述', '血钠-异常请描述', '血钾', '血钾-临床意义', '血钾-临床意义_描述',
       '血钾-异常请描述', '血氯', '血氯-临床意义', '血氯-临床意义_描述', '血氯-异常请描述', '甘油三酯',
       '甘油三酯-临床意义', '甘油三酯-临床意义_描述', '甘油三酯-异常请描述', '胆固醇', '胆固醇-临床意义',
 

In [174]:
# 提取有效字段
df_血生化_1=df_血生化_1[['访视','受试者编号','检查日期','总胆红素（TBil）','直接胆红素（DBil）','白蛋白','前白蛋白',
                  'ALT','AST','GGT','AKP','尿素氮（BUN）','肌酐（Cr）','PT','INR','血钠','血钾','血氯','甘油三酯',
                  '胆固醇','低密度脂蛋白（LDL）','高密度脂蛋白胆固醇（HDLC）']]
df_血生化_2=df_血生化_2[['访视','受试者编号','检查日期','白蛋白','HCY','NT-pro BNP','肌钙蛋白','CK-MB',
                  'ALT','AST','尿素氮（BUN）','肌酐（Cr）','血钠','血钾','血氯','甘油三酯',
                  '胆固醇','低密度脂蛋白（LDL）','高密度脂蛋白胆固醇（HDLC）']]

In [175]:
df_血生化_1.shape

(1173, 22)

In [176]:
df_血生化_1.head(1)

,访视,受试者编号,检查日期,总胆红素（TBil）,直接胆红素（DBil）,白蛋白,前白蛋白,ALT,AST,GGT,...,肌酐（Cr）,PT,INR,血钠,血钾,血氯,甘油三酯,胆固醇,低密度脂蛋白（LDL）,高密度脂蛋白胆固醇（HDLC）
0,入院检查,90001_a,2018-03-28,12.4,0.0,43.6,NaN,51.0,51.0,71.0,...,62.5,10.4,0.96,142.0,4.53,105.0,1.18,3.72,1.83,1.59


#### 入院检查

In [177]:
# 入院血生化检查
df_血生化_1_inp=df_血生化_1[df_血生化_1['访视'].str.contains('入院检查')]
df_血生化_1_inp=df_血生化_1_inp.reset_index(drop=True)
df_血生化_2_inp=df_血生化_2[df_血生化_2['访视'].str.contains('入院检查')]
df_血生化_2_inp=df_血生化_2_inp.reset_index(drop=True)

In [178]:
# 合并血生化检查
df_血生化_inp=pd.concat([df_血生化_1_inp,df_血生化_2_inp],axis=0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [179]:
df_血生化_inp.shape

(741, 26)

In [180]:
df_血生化_inp.columns

Index(['AKP', 'ALT', 'AST', 'CK-MB', 'GGT', 'HCY', 'INR', 'NT-pro BNP', 'PT',
       '低密度脂蛋白（LDL）', '前白蛋白', '受试者编号', '尿素氮（BUN）', '总胆红素（TBil）', '检查日期',
       '甘油三酯', '白蛋白', '直接胆红素（DBil）', '肌酐（Cr）', '肌钙蛋白', '胆固醇', '血氯', '血钠', '血钾',
       '访视', '高密度脂蛋白胆固醇（HDLC）'],
      dtype='object')

In [181]:
df_血生化_inp.to_excel(project_path+'/data/processed_data/检测数据/df_血生化_入院检查.xlsx')

#### 出院检查

In [182]:
# 出院血生化检查
df_血生化_1_oup=df_血生化_1[df_血生化_1['访视'].str.contains('出院前检查')]
df_血生化_1_oup=df_血生化_1_oup.reset_index(drop=True)
df_血生化_2_oup=df_血生化_2[df_血生化_2['访视'].str.contains('出院前检查')]
df_血生化_2_oup=df_血生化_2_oup.reset_index(drop=True)
# 合并血生化检查
df_血生化_oup=pd.concat([df_血生化_1_oup,df_血生化_2_oup],axis=0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [183]:
df_血生化_oup.shape

(626, 26)

In [184]:
df_血生化_oup.columns

Index(['AKP', 'ALT', 'AST', 'CK-MB', 'GGT', 'HCY', 'INR', 'NT-pro BNP', 'PT',
       '低密度脂蛋白（LDL）', '前白蛋白', '受试者编号', '尿素氮（BUN）', '总胆红素（TBil）', '检查日期',
       '甘油三酯', '白蛋白', '直接胆红素（DBil）', '肌酐（Cr）', '肌钙蛋白', '胆固醇', '血氯', '血钠', '血钾',
       '访视', '高密度脂蛋白胆固醇（HDLC）'],
      dtype='object')

In [185]:
df_血生化_oup.to_excel(project_path+'/data/processed_data/检测数据/df_血生化_出院检查.xlsx')

#### 修改列名

In [186]:
# 删除访视
df_血生化_inp=df_血生化_inp.drop(['访视','PT','INR'],axis=1) # 避免与凝血指标冲突
df_血生化_oup=df_血生化_oup.drop(['访视','PT','INR'],axis=1)
# 修改列名
protect_col=['受试者编号']
for i in [c for c in df_血生化_inp.columns if c not in protect_col]:
    df_血生化_inp.rename(columns={i:i + '_入院'},inplace=True)
for i in [c for c in df_血生化_oup.columns if c not in protect_col]:
    df_血生化_oup.rename(columns={i:i+'_出院'},inplace=True)  
df_test_血生化=pd.merge(df_血生化_inp,df_血生化_oup,on='受试者编号',how='outer')

In [187]:
df_test_血生化.shape

(741, 45)

In [188]:
df_test_血生化.head(1)

,AKP_入院,ALT_入院,AST_入院,CK-MB_入院,GGT_入院,HCY_入院,NT-pro BNP_入院,低密度脂蛋白（LDL）_入院,前白蛋白_入院,受试者编号,...,甘油三酯_出院,白蛋白_出院,直接胆红素（DBil）_出院,肌酐（Cr）_出院,肌钙蛋白_出院,胆固醇_出院,血氯_出院,血钠_出院,血钾_出院,高密度脂蛋白胆固醇（HDLC）_出院
0,NaN,51.0,51.0,NaN,71.0,NaN,NaN,1.83,NaN,90001_a,...,NaN,35.6,2.3,53.0,NaN,NaN,101.4,142.0,3.47,NaN


In [189]:
df_test_血生化.to_excel(project_path+'/data/processed_data/df_1.10.3.3_血生化_检查.xlsx')

#### 并入血生化检查

In [190]:
df_test_血生化.head()

,AKP_入院,ALT_入院,AST_入院,CK-MB_入院,GGT_入院,HCY_入院,NT-pro BNP_入院,低密度脂蛋白（LDL）_入院,前白蛋白_入院,受试者编号,...,甘油三酯_出院,白蛋白_出院,直接胆红素（DBil）_出院,肌酐（Cr）_出院,肌钙蛋白_出院,胆固醇_出院,血氯_出院,血钠_出院,血钾_出院,高密度脂蛋白胆固醇（HDLC）_出院
0,NaN,51.0,51.0,NaN,71.0,NaN,NaN,1.83,NaN,90001_a,...,NaN,35.6,2.3,53.0,NaN,NaN,101.4,142.0,3.47,NaN
1,NaN,83.0,100.0,NaN,43.0,NaN,NaN,NaN,NaN,90003_a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,9.0,17.0,NaN,33.0,NaN,NaN,3.12,253.0,90004_a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,21.0,31.0,NaN,24.0,NaN,NaN,4.27,391.0,90005_a,...,NaN,35.2,2.9,1095.0,NaN,NaN,93.0,136.9,2.92,NaN
4,NaN,17.0,16.0,NaN,16.0,NaN,NaN,4.48,191.0,90006_a,...,NaN,40.8,5.1,55.0,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
# 删除检查日期
df_test_血生化=df_test_血生化.drop(['检查日期_入院','检查日期_出院'],axis=1)
# 并入出入院-血生化检查
df_SUP_test=pd.merge(df_SUP_test,df_test_血生化,on='受试者编号',how='left')

In [192]:
df_SUP_test.shape

(211, 110)

In [193]:
df_SUP_test.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病',
       ...
       '甘油三酯_出院', '白蛋白_出院', '直接胆红素（DBil）_出院', '肌酐（Cr）_出院', '肌钙蛋白_出院', '胆固醇_出院',
       '血氯_出院', '血钠_出院', '血钾_出院', '高密度脂蛋白胆固醇（HDLC）_出院'],
      dtype='object', length=110)

### 凝血

In [194]:
# 其他科-凝血检查
df_凝血_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='LBC_凝血功能',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
# 标记受试者编号
df_凝血_1['受试者编号']=df_凝血_1['受试者编号'].apply(lambda x: str(x)+'_a')
# 心内科-凝血检查
df_凝血_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='LBC_凝血功能',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
# 标记受试者编号
df_凝血_2['受试者编号']=df_凝血_2['受试者编号'].apply(lambda x: str(x)+'_b')

In [195]:
df_凝血_1.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', 'PT', 'PT-临床意义',
       'PT-临床意义_描述', 'PT-异常请描述', 'INR', 'INR-临床意义', 'INR-临床意义_描述', 'INR-异常请描述',
       'APTT', 'APTT-临床意义', 'APTT-临床意义_描述', 'APTT-异常请描述', '原始录入日期', '最后修改日期'],
      dtype='object')

In [196]:
df_凝血_2.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '检查日期', 'PT', 'PT-临床意义',
       'PT-临床意义_描述', 'PT-异常请描述', 'INR', 'INR-临床意义', 'INR-临床意义_描述', 'INR-异常请描述',
       'APTT', 'APTT-临床意义', 'APTT-临床意义_描述', 'APTT-异常请描述', '原始录入日期', '最后修改日期'],
      dtype='object')

In [197]:
# 提取有效字段
df_凝血_1=df_凝血_1[['访视','受试者编号','检查日期','PT','INR','APTT']]
# 提取有效字段
df_凝血_2=df_凝血_2[['访视','受试者编号','检查日期','PT','INR','APTT']]

In [198]:
df_凝血_1.shape

(1167, 6)

In [199]:
df_凝血_1.head()

,访视,受试者编号,检查日期,PT,INR,APTT
0,入院检查,90001_a,2018-03-28,10.4,0.96,28.0
1,出院前检查,90001_a,2018-04-10,11.8,1.07,28.3
2,入院检查,90003_a,2018-01-31,17.6,1.63,42.0
3,出院前检查,90003_a,NaN,NaN,NaN,NaN
4,入院检查,90004_a,NaN,NaN,NaN,NaN


#### 入院检查

In [200]:
# 出入院血常规检测
df_凝血_1_inp=df_凝血_1[df_凝血_1['访视'].str.contains('入院检查')]
df_凝血_1_inp=df_凝血_1_inp.reset_index(drop=True)
df_凝血_2_inp=df_凝血_2[df_凝血_2['访视'].str.contains('入院检查')]
df_凝血_2_inp=df_凝血_2_inp.reset_index(drop=True)
# 合并凝血检查
df_凝血_inp=pd.concat([df_凝血_1_inp,df_凝血_2_inp],axis=0)

In [201]:
df_凝血_inp.shape

(741, 6)

In [202]:
df_凝血_inp.head()

,访视,受试者编号,检查日期,PT,INR,APTT
0,入院检查,90001_a,2018-03-28,10.4,0.96,28.0
1,入院检查,90003_a,2018-01-31,17.6,1.63,42.0
2,入院检查,90004_a,NaN,NaN,NaN,NaN
3,入院检查,90005_a,2018-02-07,11.9,1.10,27.7
4,入院检查,90006_a,NaN,NaN,NaN,NaN


In [203]:
df_凝血_inp.to_excel(project_path+'/data/processed_data/检测数据/df_凝血_入院记录.xlsx')

#### 出院检查

In [204]:
# 出院凝血检查
df_凝血_1_oup=df_凝血_1[df_凝血_1['访视'].str.contains('出院前检查')]
df_凝血_1_oup=df_凝血_1_oup.reset_index(drop=True)
df_凝血_2_oup=df_凝血_2[df_凝血_2['访视'].str.contains('出院前检查')]
df_凝血_2_oup=df_凝血_2_oup.reset_index(drop=True)
# 合并凝血检查
df_凝血_oup=pd.concat([df_凝血_1_oup,df_凝血_2_oup],axis=0)

In [205]:
df_凝血_oup.shape

(617, 6)

In [206]:
df_凝血_oup.head()

,访视,受试者编号,检查日期,PT,INR,APTT
0,出院前检查,90001_a,2018-04-10,11.8,1.07,28.3
1,出院前检查,90003_a,NaN,NaN,NaN,NaN
2,出院前检查,90004_a,NaN,NaN,NaN,NaN
3,出院前检查,90005_a,NaN,NaN,NaN,NaN
4,出院前检查,90006_a,2018-02-19,10.7,0.99,31.9


In [207]:
df_凝血_oup.to_excel(project_path+'/data/processed_data/检测数据/df_凝血_出院记录.xlsx')

#### 修改列名

In [208]:
# 删除访视变量
df_凝血_inp=df_凝血_inp.drop(['访视'],axis=1)
df_凝血_oup=df_凝血_oup.drop(['访视'],axis=1)
# 修改列名
for i in df_凝血_inp.columns[1:]:
    df_凝血_inp.rename(columns={i:i + '_入院'},inplace=True)
for i in df_凝血_oup.columns[1:]:
    df_凝血_oup.rename(columns={i:i+'_出院'},inplace=True)  
df_test_凝血=pd.merge(df_凝血_inp,df_凝血_oup,on='受试者编号',how='outer')

In [209]:
df_test_凝血.head()

,受试者编号,检查日期_入院,PT_入院,INR_入院,APTT_入院,检查日期_出院,PT_出院,INR_出院,APTT_出院
0,90001_a,2018-03-28,10.4,0.96,28.0,2018-04-10,11.8,1.07,28.3
1,90003_a,2018-01-31,17.6,1.63,42.0,NaN,NaN,NaN,NaN
2,90004_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90005_a,2018-02-07,11.9,1.10,27.7,NaN,NaN,NaN,NaN
4,90006_a,NaN,NaN,NaN,NaN,2018-02-19,10.7,0.99,31.9


In [210]:
df_test_凝血.shape

(741, 9)

In [211]:
df_test_凝血.to_excel(project_path+'/data/processed_data/df_1.10.4.3_凝血_检查.xlsx')

#### 并入凝血检查

In [212]:
df_test_凝血.head()

,受试者编号,检查日期_入院,PT_入院,INR_入院,APTT_入院,检查日期_出院,PT_出院,INR_出院,APTT_出院
0,90001_a,2018-03-28,10.4,0.96,28.0,2018-04-10,11.8,1.07,28.3
1,90003_a,2018-01-31,17.6,1.63,42.0,NaN,NaN,NaN,NaN
2,90004_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90005_a,2018-02-07,11.9,1.10,27.7,NaN,NaN,NaN,NaN
4,90006_a,NaN,NaN,NaN,NaN,2018-02-19,10.7,0.99,31.9


In [213]:
# 删除检查日期
df_test_凝血=df_test_凝血.drop(['检查日期_入院','检查日期_出院'],axis=1)
# 并入出入院-凝血检查
df_SUP_test=pd.merge(df_SUP_test,df_test_凝血,on='受试者编号',how='left')

In [214]:
df_SUP_test.shape

(211, 116)

### 肿瘤指标

In [215]:
# 其他科-肿瘤检查
df_肿瘤_1 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-其他科1021.xlsx',
                            sheet_name='TM_肿瘤标记物',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行
# 标记受试者编号
df_肿瘤_1['受试者编号']=df_肿瘤_1['受试者编号'].apply(lambda x: str(x)+'_a')
# 心内科-肿瘤检查
df_肿瘤_2 = pd.read_excel(project_path+'/data/raw_data/导出全部CRF表数据-心内科1021.xlsx',
                            sheet_name='TM_肿瘤标记物',
                           header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第几行)
# 标记受试者编号
df_肿瘤_2['受试者编号']=df_肿瘤_2['受试者编号'].apply(lambda x: str(x)+'_b')

In [216]:
df_肿瘤_1.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '编号', '检查日期', '检查项目', '检查项目_描述',
       '检查结果', '检查结果单位', '检查结果标准值', '检查结果标准单位', '临床意义', '临床意义_描述', '若为异常请描述',
       '原始录入日期', '最后修改日期'],
      dtype='object')

In [217]:
df_肿瘤_2.columns

Index(['方案号', '中心标识', '中心名称', '受试者编号', '访视', '编号', '检查日期', '检查项目', '检查项目_描述',
       '检查结果', '检查结果单位', '检查结果标准值', '检查结果标准单位', '临床意义', '临床意义_描述', '若为异常请描述',
       '原始录入日期', '最后修改日期'],
      dtype='object')

In [218]:
# 提取有效字段
df_肿瘤_1=df_肿瘤_1[['访视','受试者编号','检查日期','检查项目_描述','检查结果']]
# 提取有效字段
df_肿瘤_2=df_肿瘤_2[['访视','受试者编号','检查日期','检查项目_描述','检查结果']]

In [219]:
df_肿瘤_1.shape

(3215, 5)

In [220]:
df_肿瘤_1.head()

,访视,受试者编号,检查日期,检查项目_描述,检查结果
0,出院前检查,90001_a,2018-04-04,AFP,1.48
1,入院检查,90005_a,2018-02-08,AFP,3.00
2,入院检查,90005_a,2018-02-08,CEA,1.45
3,入院检查,90005_a,2018-02-08,CA-199,16.70
4,入院检查,90005_a,2018-02-08,TPSA,7.31


#### 入院检查

In [221]:
# 出入院血常规检测
df_肿瘤_1_inp=df_肿瘤_1[df_肿瘤_1['访视'].str.contains('入院检查')]
df_肿瘤_1_inp=df_肿瘤_1_inp.reset_index(drop=True)
df_肿瘤_2_inp=df_肿瘤_2[df_肿瘤_2['访视'].str.contains('入院检查')]
df_肿瘤_2_inp=df_肿瘤_2_inp.reset_index(drop=True)
# 合并肿瘤检查
df_肿瘤_inp=pd.concat([df_肿瘤_1_inp,df_肿瘤_2_inp],axis=0)

In [222]:
df_肿瘤_inp.shape

(3228, 5)

In [223]:
df_肿瘤_inp.head()

,访视,受试者编号,检查日期,检查项目_描述,检查结果
0,入院检查,90005_a,2018-02-08,AFP,3.00
1,入院检查,90005_a,2018-02-08,CEA,1.45
2,入院检查,90005_a,2018-02-08,CA-199,16.70
3,入院检查,90005_a,2018-02-08,TPSA,7.31
4,入院检查,90005_a,2018-02-08,CA-125,13.40


In [224]:
# 数据透视表-检测项转置为表头
df_肿瘤_inp=df_肿瘤_inp.pivot_table(values='检查结果',index=['受试者编号'],columns='检查项目_描述').reset_index()

In [225]:
df_肿瘤_inp.shape

(488, 13)

In [226]:
df_肿瘤_inp.head()

检查项目_描述,受试者编号,AFP,CA-125,CA-153,CA-199,CA-242,CA-50,CA-724,CEA,Cyfra211,FPSA,NSE,TPSA
0,90005_a,3.00,13.40,NaN,16.70,NaN,NaN,NaN,1.45,NaN,1.87,17.10,7.31
1,90005_b,NaN,76.80,14.70,11.40,4.77,NaN,1.50,NaN,NaN,NaN,NaN,NaN
2,90006_a,2.51,11.45,48.16,14.98,4.22,NaN,NaN,1.83,NaN,NaN,13.15,NaN
3,90007_a,3.92,11.63,15.73,20.10,6.51,NaN,2.43,2.75,5.26,NaN,12.57,NaN
4,90008_a,3.05,6.49,5.42,1.79,4.13,NaN,5.22,2.52,1.73,0.21,9.98,0.47


In [227]:
df_肿瘤_inp.to_excel(project_path+'/data/processed_data/检测数据/df_肿瘤_入院检查.xlsx')

#### 出院检查

In [228]:
# 出院肿瘤检查
df_肿瘤_1_oup=df_肿瘤_1[df_肿瘤_1['访视'].str.contains('出院前检查')]
df_肿瘤_1_oup=df_肿瘤_1_oup.reset_index(drop=True)
df_肿瘤_2_oup=df_肿瘤_2[df_肿瘤_2['访视'].str.contains('出院前检查')]
df_肿瘤_2_oup=df_肿瘤_2_oup.reset_index(drop=True)
# 合并肿瘤检查
df_肿瘤_oup=pd.concat([df_肿瘤_1_oup,df_肿瘤_2_oup],axis=0)

In [229]:
df_肿瘤_oup.shape

(14, 5)

In [230]:
df_肿瘤_oup.head()

,访视,受试者编号,检查日期,检查项目_描述,检查结果
0,出院前检查,90001_a,2018-04-04,AFP,1.48
1,出院前检查,90034_a,2018-03-01,CA-50,10.97
2,出院前检查,90037_a,2018-03-01,AFP,2.74
3,出院前检查,90037_a,2018-03-01,CEA,2.62
4,出院前检查,90037_a,2018-03-01,CA-199,25.20


In [231]:
# 数据透视表-检测项转置为表头
df_肿瘤_oup=df_肿瘤_oup.pivot_table(values='检查结果',index=['受试者编号'],columns='检查项目_描述').reset_index()

In [232]:
df_肿瘤_oup.shape

(5, 9)

In [233]:
df_肿瘤_oup.head()

检查项目_描述,受试者编号,AFP,CA-125,CA-153,CA-199,CA-242,CA-50,CA-724,CEA
0,90001_a,1.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90034_a,NaN,NaN,NaN,NaN,NaN,10.97,NaN,NaN
2,90034_b,NaN,271.00,10.60,14.2,7.89,NaN,1.6,NaN
3,90037_a,2.74,7.63,8.21,25.2,11.05,NaN,NaN,2.62
4,90052_a,NaN,NaN,NaN,NaN,NaN,7.41,NaN,NaN


In [234]:
df_肿瘤_oup.columns

Index(['受试者编号', 'AFP', 'CA-125', 'CA-153', 'CA-199', 'CA-242', 'CA-50',
       'CA-724', 'CEA'],
      dtype='object', name='检查项目_描述')

In [235]:
df_肿瘤_oup.to_excel(project_path+'/data/processed_data/检测数据/df_肿瘤_出院检查.xlsx')

#### 修改列名

In [236]:
# 修改列名
for i in df_肿瘤_inp.columns[1:]:
    df_肿瘤_inp.rename(columns={i:i + '_入院'},inplace=True)
for i in df_肿瘤_oup.columns[1:]:
    df_肿瘤_oup.rename(columns={i:i+'_出院'},inplace=True) 
    

In [237]:
df_test_肿瘤=pd.merge(df_肿瘤_inp,df_肿瘤_oup,on='受试者编号',how='outer').reset_index(drop=True)

In [238]:
df_test_肿瘤.shape

(491, 21)

In [239]:
df_test_肿瘤.columns

Index(['受试者编号', 'AFP_入院', 'CA-125_入院', 'CA-153_入院', 'CA-199_入院', 'CA-242_入院',
       'CA-50_入院', 'CA-724_入院', 'CEA_入院', 'Cyfra211_入院', 'FPSA_入院', 'NSE_入院',
       'TPSA_入院', 'AFP_出院', 'CA-125_出院', 'CA-153_出院', 'CA-199_出院', 'CA-242_出院',
       'CA-50_出院', 'CA-724_出院', 'CEA_出院'],
      dtype='object', name='检查项目_描述')

In [240]:
df_test_肿瘤.head()

检查项目_描述,受试者编号,AFP_入院,CA-125_入院,CA-153_入院,CA-199_入院,CA-242_入院,CA-50_入院,CA-724_入院,CEA_入院,Cyfra211_入院,...,NSE_入院,TPSA_入院,AFP_出院,CA-125_出院,CA-153_出院,CA-199_出院,CA-242_出院,CA-50_出院,CA-724_出院,CEA_出院
0,90005_a,3.00,13.40,NaN,16.70,NaN,NaN,NaN,1.45,NaN,...,17.10,7.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90005_b,NaN,76.80,14.70,11.40,4.77,NaN,1.50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90006_a,2.51,11.45,48.16,14.98,4.22,NaN,NaN,1.83,NaN,...,13.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90007_a,3.92,11.63,15.73,20.10,6.51,NaN,2.43,2.75,5.26,...,12.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90008_a,3.05,6.49,5.42,1.79,4.13,NaN,5.22,2.52,1.73,...,9.98,0.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
df_test_肿瘤.to_excel(project_path+'/data/processed_data/df_1.10.5.3_肿瘤_检查.xlsx')

#### 并入肿瘤指标

In [242]:
# 并入出入院-肿瘤检查
df_SUP_test=pd.merge(df_SUP_test,df_test_肿瘤,on='受试者编号',how='left')

In [243]:
df_SUP_test.shape

(211, 136)

In [244]:
df_SUP_test.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDOSFRQ',
       'EXDSTXT', '日剂量', '冠心病',
       ...
       'NSE_入院', 'TPSA_入院', 'AFP_出院', 'CA-125_出院', 'CA-153_出院', 'CA-199_出院',
       'CA-242_出院', 'CA-50_出院', 'CA-724_出院', 'CEA_出院'],
      dtype='object', length=136)

In [245]:
df_SUP_test.to_excel(project_path+'/data/processed_data/df_1.10.5.4_并入检测指标.xlsx')

### 删除缺失50%的变量

In [246]:
# 删除列缺失50%的变量
for i in df_SUP_test.columns:
    if df_SUP_test[i].isnull().sum()/df_SUP_test.shape[0] >= 0.5:
        print(i)
        del df_SUP_test[i]

红细胞_出院
血红蛋白_出院
白细胞_出院
中性粒细胞计数_出院
血小板_出院
红细胞压积_出院
尿白细胞（LEU）_入院
尿红细胞（ERY）_入院
尿蛋白（PRO）_出院
尿白细胞（LEU）_出院
尿红细胞（ERY）_出院
AKP_入院
CK-MB_入院
HCY_入院
NT-pro BNP_入院
肌钙蛋白_入院
AKP_出院
ALT_出院
AST_出院
CK-MB_出院
GGT_出院
HCY_出院
NT-pro BNP_出院
低密度脂蛋白（LDL）_出院
前白蛋白_出院
尿素氮（BUN）_出院
总胆红素（TBil）_出院
甘油三酯_出院
白蛋白_出院
直接胆红素（DBil）_出院
肌酐（Cr）_出院
肌钙蛋白_出院
胆固醇_出院
血氯_出院
血钠_出院
血钾_出院
高密度脂蛋白胆固醇（HDLC）_出院
PT_出院
INR_出院
APTT_出院
AFP_入院
CA-125_入院
CA-153_入院
CA-199_入院
CA-242_入院
CA-50_入院
CA-724_入院
CEA_入院
Cyfra211_入院
FPSA_入院
NSE_入院
TPSA_入院
AFP_出院
CA-125_出院
CA-153_出院
CA-199_出院
CA-242_出院
CA-50_出院
CA-724_出院
CEA_出院


In [247]:
# 删除行缺失50%的变量
df_SUP_test=df_SUP_test.reset_index(drop=True)
for i in range(df_SUP_test.shape[0]):
    if df_SUP_test.loc[i].isnull().sum()/df_SUP_test.loc[i].shape[0] >= 0.5:
        print(i)
        df_SUP_test.drop([i],axis=0,inplace=True)

### 删除分类不平衡数据

In [248]:
# 保护目标变量、tdm数据
protect_col=['主要疾病转归_描述','冠心病','心肌埂塞','冠心病_心梗']
col=[i for i in df_SUP_test.columns if i not in protect_col]
for i in col:
    if df_SUP_test[i].nunique() < 2:
        print(i)
        del df_SUP_test[i]
        continue
    if df_SUP_test[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_SUP_test[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_SUP_test.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_SUP_test[i]

EXDOSFRQ
头晕
急性冠脉综合征
肾功能不全
尿毒症
脓毒血症
肺水肿
肺气肿
呼吸衰竭
胸腔积液
尿路感染
腹膜透析
肝功能不全
高脂血症
慢性支气管炎
胆囊结石
早搏
急性肾损伤
心血管疾病_label
消化系统疾病_label
危重症疾病_label
family_history
allergy_history
抗凝药


### 人工删除不重要的列

In [249]:
# 人工删除不重要变量
df_SUP_test=df_SUP_test.drop(['尿蛋白（PRO）_入院'],axis=1)
# # 删除特殊字符
# df_model = filter_spec_character(df_model,'尿蛋白（PRO）_入院')    
# df_model['尿蛋白（PRO）_入院']=df_model['尿蛋白（PRO）_入院'].apply(lambda x: 0.1 if re.match('阴性|隐性', str(x)) else 
#                                                                                 np.nan if x=='nan' else x)

In [250]:
df_SUP_test.shape

(211, 51)

In [251]:
df_SUP_test.to_excel(project_path+'/data/processed_data/df_1.10.6_人工删除不重要列.xlsx')

## 划分高低剂量组

In [252]:
'''
按照40和60划分高地剂量组
'''

'\n按照40和60划分高地剂量组\n'

In [253]:
df_SUP_group=df_SUP_test.copy().reset_index(drop=True)

In [254]:
df_SUP_group.日剂量.value_counts()

60    116
40     95
Name: 日剂量, dtype: int64

In [255]:
# 划分高低剂量组
df_SUP_group['高低剂量组']=df_SUP_group.日剂量.apply(lambda x: 1 if x==60 else 0)

In [256]:
df_SUP_group['高低剂量组'].value_counts()

1    116
0     95
Name: 高低剂量组, dtype: int64

In [257]:
df_SUP_group.to_csv(project_path+'/data/processed_data/df_1.11_划分高低剂量组.csv')

In [318]:
df_SUP_group.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDSTXT',
       '日剂量', '冠心病', '肺部感染', '心功能不全', '脑梗死', '房颤', '心肌梗塞', '呼吸系统疾病_label',
       '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label', '住院日期', '完成日期',
       '住院时长', '年龄', '性别', '基础疾病', '降脂药', '降血小板药', '红细胞_入院', '血红蛋白_入院',
       '白细胞_入院', '中性粒细胞计数_入院', '血小板_入院', '红细胞压积_入院', 'ALT_入院', 'AST_入院',
       'GGT_入院', '低密度脂蛋白（LDL）_入院', '前白蛋白_入院', '尿素氮（BUN）_入院', '总胆红素（TBil）_入院',
       '甘油三酯_入院', '白蛋白_入院', '直接胆红素（DBil）_入院', '肌酐（Cr）_入院', '胆固醇_入院', '血氯_入院',
       '血钠_入院', '血钾_入院', '高密度脂蛋白胆固醇（HDLC）_入院', 'PT_入院', 'INR_入院', 'APTT_入院',
       '高低剂量组', 'data_source'],
      dtype='object')

# 对照组数据处理

## 读入患者基本信息

In [319]:
df_patient = pd.read_excel(project_path+'/data/raw_data/生脉注射液空白对照组提取数据-20220407.xlsx',
                            sheet_name='患者基本信息',)
#                            header=1,)  # 指定第2行作为标题行
#                            skiprows=[1,1,1])  # 跳过第1行

In [320]:
df_patient.columns

Index(['patient_id', 'inp_no', 'gender', 'race', 'birthplace', 'birth_year',
       'height', 'weight', 'smok_his', 'aller_his', 'drug_his', 'adr_his',
       'marriage', 'med_history', 'PatientNo', 'Cost_type', 'Insurance_type'],
      dtype='object')

In [321]:
df_patient['gender'].value_counts()

1    392
2    379
9      1
Name: gender, dtype: int64

In [322]:
# 提取有效字段
df_patient = df_patient[['PatientNo','gender','birth_year']]
# 性别
df_patient['gender'] = df_patient['gender'].apply(lambda x: 0 if x==1 else 1)

In [323]:
df_patient['gender'].value_counts()

0    392
1    380
Name: gender, dtype: int64

In [324]:
df_patient.shape

(772, 3)

In [325]:
df_patient.head()

,PatientNo,gender,birth_year
0,20435493,0,1948-10-23 00:00:00
1,580743,0,1940-03-15 00:00:00
2,22248256,1,1928-10-15 00:00:00
3,16782625,1,1951-08-24 00:00:00
4,23344656,1,1960-08-20 00:00:00


## 读入住院记录

In [496]:
df_inp = pd.read_excel(project_path+'/data/raw_data/生脉注射液空白对照组提取数据-20220407.xlsx',
                            sheet_name='住院记录',)

In [497]:
df_inp.columns

Index(['inp_record_id', 'patient_id', 'case_no', 'inp_no', 'outp_no',
       'adm_date', 'adm_dept_code', 'care_area', 'diagnosis_code',
       'diagnosis_code_type', 'diagnosis_name', 'bed_no', 'condition_status',
       'doctor_code', 'dis_date', 'dis_dept_code', 'dis_result', 'DAY_TOTAL'],
      dtype='object')

In [498]:
# 提取有效字段
df_inp = df_inp[['inp_record_id','patient_id','adm_date','dis_date','inp_no']]

In [499]:
df_inp.shape

(802, 5)

In [500]:
df_inp.head()

,inp_record_id,patient_id,adm_date,dis_date,inp_no
0,107530,20435493,2015-10-21 10:46:01,2015-11-26 08:30:00,C16178
1,120141,580743,2015-08-31 12:45:02,2015-09-06 16:30:00,C02904
2,121746,22248256,2015-08-26 09:17:30,2015-09-06 16:30:00,C01575
3,122894,16782625,2015-08-24 14:51:10,2015-09-06 16:30:00,C01093
4,16267,23344656,2016-08-17 15:59:44,1900-01-02 00:00:00,D03078


### 计算住院时长

In [501]:
df_inp['住院时长'] = (pd.to_datetime(df_inp['dis_date']) - pd.to_datetime(df_inp['adm_date']))
df_inp['住院时长'] = df_inp['住院时长'].apply(lambda x: x.days + 1)

In [502]:
df_inp['住院时长'].value_counts()

 8        99
 6        94
 7        94
 4        72
 9        54
 5        54
 10       49
 11       49
 3        47
 12       29
 2        28
 14       24
 15       19
 13       16
 1         7
 19        7
-42612     5
 17        5
 18        5
-43151     4
 20        4
 26        3
-42608     3
-42615     2
 21        2
-42611     2
 36        2
-42609     2
 16        2
-42596     2
 34        1
-42589     1
-42613     1
-42601     1
-42600     1
-42599     1
-43153     1
 215       1
-42617     1
-43150     1
-43141     1
 50        1
 37        1
 35        1
 23        1
 24        1
-42622     1
Name: 住院时长, dtype: int64

### 删除少于6天的住院时长

In [570]:
df_inp = df_inp[(df_inp['住院时长']>=9) & (df_inp['住院时长']<100)]

In [571]:
df_inp.shape

(276, 6)

In [572]:
df_inp.住院时长.value_counts()

9     54
10    49
11    49
12    29
14    24
15    19
13    16
19     7
18     5
17     5
20     4
26     3
21     2
16     2
36     2
37     1
23     1
24     1
34     1
35     1
50     1
Name: 住院时长, dtype: int64

### 并入住院时长

In [573]:
df_patient.columns

Index(['patient_id', 'gender', 'birth_year'], dtype='object')

In [574]:
df_inp.columns

Index(['inp_record_id', 'patient_id', 'adm_date', 'dis_date', 'inp_no',
       '住院时长'],
      dtype='object')

In [575]:
# 修改列名
df_patient = df_patient.rename(columns={'PatientNo':'patient_id'})
df_patient_inp = pd.merge(df_patient, df_inp,on='patient_id',how='right')

In [576]:
df_patient_inp.shape

(276, 8)

In [577]:
df_patient_inp.to_csv(project_path+'/data/processed_data/df_2.2.3_对照组_住院时长.csv')

### 计算年龄

In [578]:
df_patient_inp.head()

,patient_id,gender,birth_year,inp_record_id,adm_date,dis_date,inp_no,住院时长
0,20435493,0,1948-10-23 00:00:00,107530,2015-10-21 10:46:01,2015-11-26 08:30:00,C16178,36
1,22248256,1,1928-10-15 00:00:00,121746,2015-08-26 09:17:30,2015-09-06 16:30:00,C01575,12
2,16782625,1,1951-08-24 00:00:00,122894,2015-08-24 14:51:10,2015-09-06 16:30:00,C01093,14
3,23396619,0,1960-01-16 00:00:00,17060,2016-08-23 08:33:01,2016-08-31 16:30:00,D04623,9
4,9167605,1,1938-04-14 00:00:00,825293,2017-02-02 10:51:09,2017-02-13 16:30:00,D51365,12


In [579]:
# 提取出生年
df_patient_inp['birth_year']=df_patient_inp['birth_year'].apply(lambda x: str(x).split('-')[0])
# 计算年龄
df_patient_inp['age']=df_patient_inp['adm_date'].apply(lambda x: str(x).split('-')[0])
df_patient_inp['age']=df_patient_inp['age'].astype(int)-df_patient_inp['birth_year'].astype(int)

In [580]:
df_patient_inp.to_csv(project_path+'/data/processed_data/df_2.2.4_计算年龄.csv')

## 诊断记录

In [581]:
df_diagnose = pd.read_excel(project_path+'/data/raw_data/生脉注射液空白对照组提取数据-20220407.xlsx',
                            sheet_name='诊断记录',)

In [582]:
df_diagnose.columns

Index(['diagnostic_record_id', 'patient_id', 'inp_record_id', 'outp_record_id',
       'record_date', 'diagnostic_type', 'diagnostic_content',
       'treatment_days', 'treatment_result', 'IsPrimary'],
      dtype='object')

In [583]:
# 提取字段
df_diagnose = df_diagnose[['inp_record_id','diagnostic_content']]

In [584]:
df_diagnose.diagnostic_content.unique()

array(['急性心肌梗死', '心律失常型冠心病', '心肌梗死', '急性下壁心肌梗死', '急性非ST段抬高型心肌梗塞',
       '冠状动脉旁路术后心肌梗塞', '冠状动脉粥样硬化', '可疑冠心病观察', '急性前壁心肌梗死', '2型糖尿病'],
      dtype=object)

### 筛选冠心病和心梗

In [585]:
df_diagnose_disease = df_diagnose[df_diagnose['diagnostic_content'].str.contains('冠心病|心肌梗死|心肌梗塞')]

In [586]:
df_diagnose_disease['冠心病'] = df_diagnose_disease.diagnostic_content.apply(lambda x: 1 if re.match('.*冠心病.*',x) else 0)
df_diagnose_disease['心肌梗塞'] = df_diagnose_disease.diagnostic_content.apply(lambda x: 1 if re.match('.*[梗死梗塞].*',x) else 0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [587]:
df_diagnose_disease.to_csv(project_path+'/data/processed_data/df_2.3.1_提取冠心病和心梗.csv')

### 并入疾病诊断

In [588]:
df_patient_inp.columns

Index(['patient_id', 'gender', 'birth_year', 'inp_record_id', 'adm_date',
       'dis_date', 'inp_no', '住院时长', 'age'],
      dtype='object')

In [589]:
df_patient_inp.columns

Index(['patient_id', 'gender', 'birth_year', 'inp_record_id', 'adm_date',
       'dis_date', 'inp_no', '住院时长', 'age'],
      dtype='object')

In [590]:
df_patient_disease = pd.merge(df_patient_inp,df_diagnose_disease,on='inp_record_id',how='left')

In [591]:
df_patient_disease.to_csv(project_path+'/data/processed_data/df_2.3.2_对照组_并入疾病诊断.csv')

PermissionError: [Errno 13] Permission denied: 'D:\\PycharmProjects\\SUP_xinhua_hospital/data/processed_data/df_2.3.2_对照组_并入疾病诊断.csv'

## 住院医嘱

In [592]:
df_patient_diagnose = df_patient_disease.copy()

In [593]:
df_SUP_group.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDSTXT',
       '日剂量', '冠心病', '肺部感染', '心功能不全', '脑梗死', '房颤', '心肌梗塞', '呼吸系统疾病_label',
       '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label', '住院日期', '完成日期',
       '住院时长', '年龄', '性别', '基础疾病', '降脂药', '降血小板药', '红细胞_入院', '血红蛋白_入院',
       '白细胞_入院', '中性粒细胞计数_入院', '血小板_入院', '红细胞压积_入院', 'ALT_入院', 'AST_入院',
       'GGT_入院', '低密度脂蛋白（LDL）_入院', '前白蛋白_入院', '尿素氮（BUN）_入院', '总胆红素（TBil）_入院',
       '甘油三酯_入院', '白蛋白_入院', '直接胆红素（DBil）_入院', '肌酐（Cr）_入院', '胆固醇_入院', '血氯_入院',
       '血钠_入院', '血钾_入院', '高密度脂蛋白胆固醇（HDLC）_入院', 'PT_入院', 'INR_入院', 'APTT_入院',
       '高低剂量组', 'data_source'],
      dtype='object')

In [594]:
df_patient_diagnose.columns

Index(['patient_id', 'gender', 'birth_year', 'inp_record_id', 'adm_date',
       'dis_date', 'inp_no', '住院时长', 'age', 'diagnostic_content', '冠心病',
       '心肌梗塞'],
      dtype='object')

In [595]:
df_drug = pd.read_excel(project_path+'/data/raw_data/生脉注射液空白对照组提取数据-20220407.xlsx',
                            sheet_name='住院医嘱',)

In [596]:
# 提取字段
df_drug = df_drug[['case_no','drug_name','start_datetime','end_datetime']]

In [597]:
df_drug_降血脂药=df_drug[df_drug.drug_name.str.contains('他汀')]
df_drug_抗血小板药=df_drug[df_drug.drug_name.str.contains('阿司匹|替格瑞|硫酸氢氯')]
df_drug_抗凝药=df_drug[df_drug.drug_name.str.contains('达肝素钠注射液|纤溶酶针|依诺肝素|达肝素钠|华法林钠片')]

In [598]:
# 并入联合用药：降脂药、降血小板药、抗凝药
df_patient_diagnose['降脂药']=df_patient_diagnose.inp_record_id.apply(lambda x: 1 if x in df_drug_降血脂药.case_no.unique() else 0)
df_patient_diagnose['降血小板药']=df_patient_diagnose.inp_record_id.apply(lambda x: 1 if x in df_drug_抗血小板药.case_no.unique() else 0)
df_patient_diagnose['抗凝药']=df_patient_diagnose.inp_record_id.apply(lambda x: 1 if x in df_drug_抗凝药.case_no.unique() else 0)

In [599]:
df_patient_diagnose.head()

,patient_id,gender,birth_year,inp_record_id,adm_date,dis_date,inp_no,住院时长,age,diagnostic_content,冠心病,心肌梗塞,降脂药,降血小板药,抗凝药
0,20435493,0,1948,107530,2015-10-21 10:46:01,2015-11-26 08:30:00,C16178,36,67,急性心肌梗死,0,1,1,1,1
1,22248256,1,1928,121746,2015-08-26 09:17:30,2015-09-06 16:30:00,C01575,12,87,心律失常型冠心病,1,0,1,0,0
2,16782625,1,1951,122894,2015-08-24 14:51:10,2015-09-06 16:30:00,C01093,14,64,心律失常型冠心病,1,0,0,0,1
3,23396619,0,1960,17060,2016-08-23 08:33:01,2016-08-31 16:30:00,D04623,9,56,心律失常型冠心病,1,0,0,1,0
4,9167605,1,1938,825293,2017-02-02 10:51:09,2017-02-13 16:30:00,D51365,12,79,心律失常型冠心病,1,0,1,1,1


In [600]:
df_patient_diagnose.to_csv(project_path+'/data/processed_data/df_2.4_对照组_并入用药.csv')

## 修改列名

In [601]:
df_SUP_group.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDSTXT',
       '日剂量', '冠心病', '肺部感染', '心功能不全', '脑梗死', '房颤', '心肌梗塞', '呼吸系统疾病_label',
       '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label', '住院日期', '完成日期',
       '住院时长', '年龄', '性别', '基础疾病', '降脂药', '降血小板药', '红细胞_入院', '血红蛋白_入院',
       '白细胞_入院', '中性粒细胞计数_入院', '血小板_入院', '红细胞压积_入院', 'ALT_入院', 'AST_入院',
       'GGT_入院', '低密度脂蛋白（LDL）_入院', '前白蛋白_入院', '尿素氮（BUN）_入院', '总胆红素（TBil）_入院',
       '甘油三酯_入院', '白蛋白_入院', '直接胆红素（DBil）_入院', '肌酐（Cr）_入院', '胆固醇_入院', '血氯_入院',
       '血钠_入院', '血钾_入院', '高密度脂蛋白胆固醇（HDLC）_入院', 'PT_入院', 'INR_入院', 'APTT_入院',
       '高低剂量组', 'data_source'],
      dtype='object')

In [602]:
df_patient_diagnose.columns

Index(['patient_id', 'gender', 'birth_year', 'inp_record_id', 'adm_date',
       'dis_date', 'inp_no', '住院时长', 'age', 'diagnostic_content', '冠心病',
       '心肌梗塞', '降脂药', '降血小板药', '抗凝药'],
      dtype='object')

In [603]:
df_patient_diagnose.columns=['受试者编号','性别','出生年','住院记录号','住院日期','完成日期','住院号','住院时长', '年龄', 
                             '出院诊断', '冠心病','心肌梗塞', '降脂药', '降血小板药', '抗凝药']

In [604]:
# 提取字段
df_patient_diagnose = df_patient_diagnose[['受试者编号','出院诊断', '冠心病','心肌梗塞','住院日期','完成日期','住院时长', '年龄', '性别',
                              '降脂药', '降血小板药']]

## 删除缺失50%

In [605]:
# 删除列缺失50%的变量
for i in df_patient_diagnose.columns:
    if df_patient_diagnose[i].isnull().sum()/df_patient_diagnose.shape[0] >= 0.5:
        print(i)
        del df_patient_diagnose[i]

In [606]:
# 删除行缺失50%的变量
df_patient_diagnose=df_patient_diagnose.reset_index(drop=True)
for i in range(df_patient_diagnose.shape[0]):
    if df_patient_diagnose.loc[i].isnull().sum()/df_patient_diagnose.loc[i].shape[0] >= 0.5:
        print(i)
        df_patient_diagnose.drop([i],axis=0,inplace=True)

## 删除不平衡分类

In [607]:
# 保护目标变量、tdm数据
protect_col=['主要疾病转归_描述','冠心病','心肌梗塞','冠心病_心梗']
col=[i for i in df_patient_diagnose.columns if i not in protect_col]
for i in col:
    if df_patient_diagnose[i].nunique() < 2:
        print(i)
        del df_patient_diagnose[i]
        continue
    if df_patient_diagnose[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_patient_diagnose[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_patient_diagnose.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_patient_diagnose[i]

In [608]:
df_patient_diagnose.shape

(276, 11)

In [609]:
df_patient_diagnose.to_csv(project_path+'/data/processed_data/df_2.7_删除不平衡分类.csv')

# 合并对照组和实验组数据

In [610]:
df_SUP_group.shape

(211, 53)

In [611]:
df_SUP_group[(df_SUP_group['年龄']<=65) & (df_SUP_group['年龄']>=25) & (df_SUP_group['冠心病']==1)].住院时长.mean()

12.88888888888889

In [612]:
df_patient_diagnose[(df_patient_diagnose.年龄<=65) & (df_patient_diagnose.年龄>=25) & (df_patient_diagnose.冠心病==1)].住院时长.mean()

11.561643835616438

In [613]:
df_SUP_group.住院时长.mean()

13.407582938388625

In [614]:
df_patient_diagnose.住院时长.mean()

12.706521739130435

In [615]:
# 打标签
df_SUP_group['data_source']=1
df_patient_diagnose['data_source']=0
# 合并数据
df_SUP_control = pd.concat([df_SUP_group,df_patient_diagnose],axis=0,sort=False)

In [616]:
df_SUP_control.shape

(487, 53)

In [617]:
df_SUP_control.head()

,受试者编号,出院诊断,主要疾病转归_描述,用药开始时间,用药结束时间,用药天数,EXDSTXT,日剂量,冠心病,肺部感染,...,胆固醇_入院,血氯_入院,血钠_入院,血钾_入院,高密度脂蛋白胆固醇（HDLC）_入院,PT_入院,INR_入院,APTT_入院,高低剂量组,data_source
0,90002_b,"急性前壁心肌梗塞,冠心病killip Ⅰ级,急诊LAD-PCI术; 2型糖尿病",0.0,2019-04-24,2019-05-05,12.0,60.0,60.0,1,0.0,...,4.65,103.0,139.0,4.3,1.55,14.9,1.09,29.9,1.0,1
1,90006_b,":急性前壁心肌梗塞,killip I级,冠心病 急诊PCI术后 高血压病(2级,极高危),类...",0.0,2019-04-17,2019-04-24,8.0,60.0,60.0,1,0.0,...,NaN,105.0,140.0,3.9,1.04,15.7,1.23,38.8,1.0,1
2,90007_b,":急性下壁心肌梗塞,Killip I级,急诊RCA-PCI术后;冠心病 三支病变;2型糖尿病...",0.0,2019-05-20,2019-05-21,2.0,60.0,60.0,1,0.0,...,4.12,103.0,135.0,3.9,1.11,15.0,1.11,37.9,1.0,1
3,90008_b,"急性下壁心肌梗塞,冠心病 ,killip Ⅰ级,急诊PCI术后,甲状腺功能亢进,胆囊结石。",0.0,2019-06-26,2019-06-27,2.0,60.0,60.0,1,0.0,...,3.80,100.0,135.0,5.0,1.03,13.5,1.12,39.2,1.0,1
4,90009_b,"急性前壁心肌梗塞,killip Ⅳ级,急诊PCI术后 冠心病 心功能不全 高血压病1级,很高...",0.0,2019-04-25,2019-04-27,3.0,60.0,60.0,1,0.0,...,5.34,102.0,135.0,3.5,1.63,11.6,1.25,50.9,1.0,1


In [620]:
df_SUP_control.to_csv(project_path+'/data/processed_data/df_3_合并对照组和实验组.csv')

In [621]:
df_SUP_control.columns

Index(['受试者编号', '出院诊断', '主要疾病转归_描述', '用药开始时间', '用药结束时间', '用药天数', 'EXDSTXT',
       '日剂量', '冠心病', '肺部感染', '心功能不全', '脑梗死', '房颤', '心肌梗塞', '呼吸系统疾病_label',
       '神经系统疾病_label', '泌尿系统疾病_label', '内分泌和代谢疾病_label', '住院日期', '完成日期',
       '住院时长', '年龄', '性别', '基础疾病', '降脂药', '降血小板药', '红细胞_入院', '血红蛋白_入院',
       '白细胞_入院', '中性粒细胞计数_入院', '血小板_入院', '红细胞压积_入院', 'ALT_入院', 'AST_入院',
       'GGT_入院', '低密度脂蛋白（LDL）_入院', '前白蛋白_入院', '尿素氮（BUN）_入院', '总胆红素（TBil）_入院',
       '甘油三酯_入院', '白蛋白_入院', '直接胆红素（DBil）_入院', '肌酐（Cr）_入院', '胆固醇_入院', '血氯_入院',
       '血钠_入院', '血钾_入院', '高密度脂蛋白胆固醇（HDLC）_入院', 'PT_入院', 'INR_入院', 'APTT_入院',
       '高低剂量组', 'data_source'],
      dtype='object')

# 倾向性评分匹配